In [1]:
import pandas as pd
import numpy as np
from nba_api.stats import endpoints as nba_endpoints
from nba_api.live.nba import endpoints as nba_live_endpoints
import plotly.graph_objects as go
import plotly.io as pio
from tqdm.notebook import tqdm, trange
from pprint import pprint
from math import floor, ceil
import goldsberry

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import cmocean #http://matplotlib.org/cmocean/

In [2]:
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 500)
pio.renderers.default = 'iframe' 

In [3]:
shotchart = nba_endpoints.shotchartdetail.ShotChartDetail(
    team_id='1610612738',
    player_id='1628369',
    context_measure_simple = 'FGA',
    season_nullable = '2023-24',
    season_type_all_star = 'Regular Season'
).get_data_frames()[0]

In [ ]:
shotchart.loc[shotchart["GAME_ID"]=='0022300528']

# Data Ingestion

In [10]:
seasons = [
    f"{x}-{(x+1)-2000}" for x in range(int(2023),2025)
]

## Teams

In [11]:
teams = nba_endpoints.commonteamyears.CommonTeamYears().get_data_frames()[0][["TEAM_ID", "ABBREVIATION"]]

## Games

In [12]:
teamlogs = nba_endpoints.teamgamelogs

games = pd.DataFrame()
for s in tqdm(seasons):
    games = pd.concat(
        [
            games,
            teamlogs.TeamGameLogs(season_nullable=s).get_data_frames()[0]
        ]
    )
game_ids = games["GAME_ID"]

  0%|          | 0/2 [00:00<?, ?it/s]

In [248]:
game_ids = list(set(game_ids))

In [249]:
len(game_ids)

2460

## All Plays

In [250]:
all_plays = pd.DataFrame()
for i in tqdm(range(len(game_ids))):
    print(i) if i%100 == 0 else None
    g = game_ids[i]
    try:
        df = pd.DataFrame(
            nba_endpoints.PlayByPlayV3(game_id=g).get_dict().get("game").get("actions")
        )
        df['scoreHome'] = np.where(df['scoreHome'] == '', np.NaN, df['scoreHome']).astype(float)
        df['scoreAway'] = np.where(df['scoreAway'] == '', np.NaN, df['scoreAway']).astype(float)
        dff = df.assign(
            game_id=g,
            point_diff=lambda row: row['scoreHome'] - row['scoreAway']
        )
        all_plays = pd.concat([all_plays, dff], axis=0)
    except Exception as e:
        print(f"ERROR: Game {i+1} out of {len(game_ids)} -> {e}")

  0%|          | 0/2460 [00:00<?, ?it/s]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400


In [251]:
all_plays.game_id.nunique()

2460

In [252]:
all_plays.to_parquet("all_plays.parquet")

In [13]:
all_plays = pd.read_parquet("all_plays.parquet")

# Feature Engineering

In [14]:
all_plays

,actionNumber,clock,period,teamId,teamTricode,personId,playerName,playerNameI,xLegacy,yLegacy,shotDistance,shotResult,isFieldGoal,scoreHome,scoreAway,pointsTotal,location,description,actionType,subType,videoAvailable,shotValue,actionId,game_id,point_diff
0,2,PT12M00.00S,1,0,,0,,,0,0,0,,0,0.0,0.0,0,,Start of 1st Period (7:41 PM EST),period,start,0,0,1,0022300118,0.0
1,4,PT12M00.00S,1,1610612738,BOS,204001,Porziņģis,K. Porziņģis,0,0,0,,0,NaN,NaN,0,h,Jump Ball Porzingis vs. Turner: Tip to Brown,Jump Ball,,1,0,2,0022300118,NaN
2,7,PT11M43.00S,1,1610612738,BOS,201950,Holiday,J. Holiday,43,7,4,Made,1,2.0,0.0,2,h,Holiday 4' Finger Roll Layup (2 PTS),Made Shot,Finger Roll Layup Shot,1,2,3,0022300118,2.0
3,9,PT11M33.00S,1,1610612754,IND,1629614,Nembhard,A. Nembhard,-64,151,16,Made,1,2.0,2.0,4,v,Nembhard 16' Pullup Jump Shot (2 PTS),Made Shot,Pullup Jump shot,1,2,4,0022300118,0.0
4,10,PT11M23.00S,1,1610612738,BOS,1628401,White,D. White,15,110,11,Made,1,4.0,2.0,6,h,White 11' Driving Floating Jump Shot (2 PTS),Made Shot,Driving Floating Jump Shot,1,2,5,0022300118,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,649,PT00M07.00S,4,1610612753,ORL,203914,Harris,G. Harris,0,0,0,,0,97.0,102.0,199,v,Harris Free Throw 1 of 2 (14 PTS),Free Throw,Free Throw 1 of 2,1,0,469,0022300033,-5.0
469,650,PT00M07.00S,4,1610612753,ORL,203914,Harris,G. Harris,0,0,0,,0,97.0,103.0,200,v,Harris Free Throw 2 of 2 (15 PTS),Free Throw,Free Throw 2 of 2,1,0,470,0022300033,-6.0
470,651,PT00M01.90S,4,1610612741,CHI,203897,LaVine,Z. LaVine,60,249,26,Missed,1,NaN,NaN,0,h,MISS LaVine 26' 3PT Pullup Jump Shot,Missed Shot,Pullup Jump shot,1,3,471,0022300033,NaN
471,652,PT00M01.90S,4,1610612753,ORL,203914,Harris,G. Harris,0,0,0,,0,NaN,NaN,0,v,Harris REBOUND (Off:2 Def:4),Rebound,Unknown,1,0,472,0022300033,NaN


## Home & Away Teams

In [15]:
games["home"] = games["MATCHUP"].str.contains("vs.")
games["team_1"] = games["MATCHUP"].str.extract("(^[A-Z]{3})")
games["team_2"] = games["MATCHUP"].str.extract("([A-Z]{3}$)")

In [16]:
games["home_team"] = np.where(
    games["home"] == True,
    games["team_1"],
    games["team_2"]
)
games["away_team"] = np.where(
    games["home"] == True,
    games["team_2"],
    games["team_1"]
)

In [17]:
games[["MATCHUP", "home_team", "away_team"]]

,MATCHUP,home_team,away_team
0,DET @ SAS,SAS,DET
1,CHA @ CLE,CLE,CHA
2,TOR @ MIA,MIA,TOR
3,SAS vs. DET,SAS,DET
4,BOS vs. WAS,BOS,WAS
...,...,...,...
2455,CHA @ HOU,HOU,CHA
2456,BOS vs. NYK,BOS,NYK
2457,LAL vs. MIN,LAL,MIN
2458,NYK @ BOS,BOS,NYK


## Seconds Elapsed

In [18]:
all_plays["minutes"] = all_plays["clock"].str.extract(
    "[PT]{2}([0-9]{2})M([0-9]{2}).[0-9]{2}S")[0].astype(int)
all_plays["seconds"] = all_plays["clock"].str.extract(
    "[PT]{2}([0-9]{2})M([0-9]{2}).[0-9]{2}S")[1].astype(int)
all_plays["seconds_elapsed"] = ((all_plays["period"].astype(int)-1)*12*60) + \
    ((np.where(all_plays["period"]==5,5,12)*60)-(((all_plays["minutes"])*60)+all_plays["seconds"]))

In [19]:
all_plays[["clock", "minutes", "seconds", "seconds_elapsed"]]

,clock,minutes,seconds,seconds_elapsed
0,PT12M00.00S,12,0,0
1,PT12M00.00S,12,0,0
2,PT11M43.00S,11,43,17
3,PT11M33.00S,11,33,27
4,PT11M23.00S,11,23,37
...,...,...,...,...
468,PT00M07.00S,0,7,2873
469,PT00M07.00S,0,7,2873
470,PT00M01.90S,0,1,2879
471,PT00M01.90S,0,1,2879


## 20-point Lead Action Number

In [20]:
game_ids_20_point_leads = all_plays.loc[
    (~pd.isnull(all_plays["scoreHome"]))
    & (all_plays["point_diff"].abs() >= 20)
].groupby("game_id").agg({
    "actionNumber": "min",
    "seconds_elapsed": "min"
}).reset_index()

game_ids_20_point_leads = game_ids_20_point_leads.merge(
    all_plays.loc[
        ~pd.isnull(all_plays["scoreHome"])
    ].groupby("game_id").agg({
        "actionNumber": "max"
    }).reset_index(),
    on="game_id"
)

game_ids_20_point_leads.columns = ["game_id", "lead_20_actionNumber", "lead_20_seconds_elapsed", "end_actionNumber"]

In [21]:
game_ids_20_point_leads

,game_id,lead_20_actionNumber,lead_20_seconds_elapsed,end_actionNumber
0,0022300003,376,1935,602
1,0022300006,426,1747,687
2,0022300010,551,2469,635
3,0022300012,428,1944,673
4,0022300014,292,1210,707
...,...,...,...,...
1166,0022401220,348,1292,795
1167,0022401222,516,2054,730
1168,0022401224,144,691,718
1169,0022401225,160,653,751


## Filter to Games With 20-point Leads

In [22]:
all_plays_fil = all_plays.loc[~pd.isnull(all_plays["point_diff"])].merge(
    game_ids_20_point_leads,
    on="game_id"
).merge(
    games[["GAME_ID", "home_team", "away_team"]].drop_duplicates(),
    left_on="game_id",
    right_on="GAME_ID"
).assign(
    leading_team=lambda row: np.where(row["point_diff"]<0, "away", "home")
).drop("GAME_ID", axis=1)

## Blown Leads

### Tied or Lost Lead

In [23]:
all_plays_fil["tied_or_lost_lead"] = np.where(
    all_plays_fil["actionNumber"] > all_plays_fil["lead_20_actionNumber"], 
    np.where(all_plays_fil["leading_team"]=="away", all_plays_fil["point_diff"] >= 0, all_plays_fil["point_diff"] <= 0),
    False
)
all_plays_fil["tied_lost_lead_game_id"] = np.where(
    all_plays_fil["tied_or_lost_lead"], all_plays_fil["game_id"], None
)

In [24]:
game_ids_tied_lost_lead = all_plays_fil.loc[all_plays_fil["tied_or_lost_lead"]].groupby("tied_lost_lead_game_id").agg({
    "actionNumber": "min",
    "seconds_elapsed": "min"
}).reset_index().rename(
    columns={
        "actionNumber": "tied_lost_lead_actionNumber",
        "seconds_elapsed": "tied_lost_lead_seconds_elapsed"
    }
).assign()

In [25]:
game_ids_tied_lost_lead.head()

,tied_lost_lead_game_id,tied_lost_lead_actionNumber,tied_lost_lead_seconds_elapsed
0,0022300012,573,2486
1,0022300033,596,2702
2,0022300039,482,1923
3,0022300045,565,2761
4,0022300052,380,1676


In [26]:
all_plays_fil = all_plays_fil.merge(
    game_ids_tied_lost_lead,
    left_on="game_id",
    right_on="tied_lost_lead_game_id",
    how="left"
).drop("tied_lost_lead_game_id_x", axis=1).rename(columns={
    "tied_lost_lead_game_id_y": "tied_lost_lead_game_id"
})

In [27]:
all_plays_fil["seconds_to_tied_lost_lead"] = np.where(
    pd.isnull(all_plays_fil["tied_lost_lead_actionNumber"]),
    np.NaN,
    all_plays_fil["tied_lost_lead_seconds_elapsed"]-all_plays_fil["lead_20_seconds_elapsed"]
)

In [28]:
all_plays_fil.loc[~pd.isnull(all_plays_fil["seconds_to_tied_lost_lead"])]

,actionNumber,clock,period,teamId,teamTricode,personId,playerName,playerNameI,xLegacy,yLegacy,shotDistance,shotResult,isFieldGoal,scoreHome,scoreAway,pointsTotal,location,description,actionType,subType,videoAvailable,shotValue,actionId,game_id,point_diff,minutes,seconds,seconds_elapsed,lead_20_actionNumber,lead_20_seconds_elapsed,end_actionNumber,home_team,away_team,leading_team,tied_or_lost_lead,tied_lost_lead_game_id,tied_lost_lead_actionNumber,tied_lost_lead_seconds_elapsed,seconds_to_tied_lost_lead
144,2,PT12M00.00S,1,0,,0,,,0,0,0,,0,0.0,0.0,0,,Start of 1st Period (8:11 PM EST),period,start,1,0,1,0022300371,0.0,12,0,0,304,1278,647,HOU,ATL,home,False,0022300371,445.0,1908.0,630.0
145,9,PT11M26.00S,1,1610612737,ATL,1629631,Hunter,D. Hunter,238,9,24,Made,1,0.0,3.0,3,v,Hunter 24' 3PT Jump Shot (3 PTS) (Murray 1 AST),Made Shot,Jump Shot,1,3,5,0022300371,-3.0,11,26,34,304,1278,647,HOU,ATL,away,False,0022300371,445.0,1908.0,630.0
146,13,PT11M13.00S,1,1610612745,HOU,1630578,Sengun,A. Sengun,0,0,0,Made,1,2.0,3.0,5,h,Sengun Tip Layup Shot (2 PTS),Made Shot,Tip Layup Shot,1,2,8,0022300371,-1.0,11,13,47,304,1278,647,HOU,ATL,away,False,0022300371,445.0,1908.0,630.0
147,20,PT10M41.00S,1,1610612745,HOU,1631095,Smith Jr.,J. Smith Jr.,-38,115,12,Made,1,4.0,3.0,7,h,Smith Jr. 12' Jump Shot (2 PTS),Made Shot,Jump Shot,1,2,15,0022300371,1.0,10,41,79,304,1278,647,HOU,ATL,home,False,0022300371,445.0,1908.0,630.0
148,24,PT10M04.00S,1,1610612737,ATL,1629631,Hunter,D. Hunter,-126,22,13,Made,1,4.0,5.0,9,v,Hunter 13' Pullup Jump Shot (5 PTS),Made Shot,Pullup Jump shot,1,2,19,0022300371,-1.0,10,4,116,304,1278,647,HOU,ATL,away,False,0022300371,445.0,1908.0,630.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149782,641,PT00M12.20S,4,1610612753,ORL,1628371,Isaac,J. Isaac,0,0,0,,0,97.0,100.0,197,v,Isaac Free Throw 1 of 2 (17 PTS),Free Throw,Free Throw 1 of 2,1,0,462,0022300033,-3.0,0,12,2868,351,1554,653,CHI,ORL,away,False,0022300033,596.0,2702.0,1148.0
149783,642,PT00M12.20S,4,1610612753,ORL,1628371,Isaac,J. Isaac,0,0,0,,0,97.0,101.0,198,v,Isaac Free Throw 2 of 2 (18 PTS),Free Throw,Free Throw 2 of 2,1,0,463,0022300033,-4.0,0,12,2868,351,1554,653,CHI,ORL,away,False,0022300033,596.0,2702.0,1148.0
149784,649,PT00M07.00S,4,1610612753,ORL,203914,Harris,G. Harris,0,0,0,,0,97.0,102.0,199,v,Harris Free Throw 1 of 2 (14 PTS),Free Throw,Free Throw 1 of 2,1,0,469,0022300033,-5.0,0,7,2873,351,1554,653,CHI,ORL,away,False,0022300033,596.0,2702.0,1148.0
149785,650,PT00M07.00S,4,1610612753,ORL,203914,Harris,G. Harris,0,0,0,,0,97.0,103.0,200,v,Harris Free Throw 2 of 2 (15 PTS),Free Throw,Free Throw 2 of 2,1,0,470,0022300033,-6.0,0,7,2873,351,1554,653,CHI,ORL,away,False,0022300033,596.0,2702.0,1148.0


In [29]:
all_plays_fil.drop("tied_or_lost_lead", axis=1, inplace=True)

### Within 2 Possessions
2 Possession game with >= 1 minute left in game

In [30]:
all_plays_fil["within_2_possessions"] = np.where(
    all_plays_fil["actionNumber"] > all_plays_fil["lead_20_actionNumber"], 
    (np.where(all_plays_fil["leading_team"]=="away", all_plays_fil["point_diff"] >= -6, all_plays_fil["point_diff"] <= 6))
    & ((all_plays_fil["period"] < 4) | (all_plays_fil["minutes"] >= 1)),
    False
)
all_plays_fil["within_2_possessions_game_id"] = np.where(
    all_plays_fil["within_2_possessions"], all_plays_fil["game_id"], None
)

In [31]:
game_ids_within_2_possessions = all_plays_fil.loc[
    all_plays_fil["within_2_possessions"]
].groupby("within_2_possessions_game_id").agg({
    "actionNumber": "min",
    "seconds_elapsed": "min"
}).reset_index().rename(
    columns={
        "actionNumber": "within_2_possessions_actionNumber",
        "seconds_elapsed": "within_2_possessions_seconds_elapsed"
    }
)

In [32]:
game_ids_within_2_possessions

,within_2_possessions_game_id,within_2_possessions_actionNumber,within_2_possessions_seconds_elapsed
0,0022300012,502,2180
1,0022300028,335,1616
2,0022300033,558,2552
3,0022300034,499,2123
4,0022300039,438,1743
...,...,...,...
256,0022401192,327,1596
257,0022401196,419,1728
258,0022401197,444,1987
259,0022401222,576,2312


In [33]:
all_plays_fil = all_plays_fil.merge(
    game_ids_within_2_possessions,
    left_on="game_id",
    right_on="within_2_possessions_game_id",
    how="left"
).drop("within_2_possessions_game_id_x", axis=1).rename(columns={
    "within_2_possessions_game_id_y": "within_2_possessions_game_id"
})

In [34]:
all_plays_fil["seconds_to_within_2_possessions"] = np.where(
    pd.isnull(all_plays_fil["within_2_possessions_actionNumber"]),
    np.NaN,
    all_plays_fil["within_2_possessions_seconds_elapsed"]-all_plays_fil["lead_20_seconds_elapsed"]
)

In [35]:
all_plays_fil.loc[all_plays_fil["game_id"]=='0022300012']

,actionNumber,clock,period,teamId,teamTricode,personId,playerName,playerNameI,xLegacy,yLegacy,shotDistance,shotResult,isFieldGoal,scoreHome,scoreAway,pointsTotal,location,description,actionType,subType,videoAvailable,shotValue,actionId,game_id,point_diff,minutes,seconds,seconds_elapsed,lead_20_actionNumber,lead_20_seconds_elapsed,end_actionNumber,home_team,away_team,leading_team,tied_lost_lead_game_id,tied_lost_lead_actionNumber,tied_lost_lead_seconds_elapsed,seconds_to_tied_lost_lead,within_2_possessions,within_2_possessions_game_id,within_2_possessions_actionNumber,within_2_possessions_seconds_elapsed,seconds_to_within_2_possessions
41730,2,PT12M00.00S,1,0,,0,,,0,0,0,,0,0.0,0.0,0,,Start of 1st Period (8:11 PM EST),period,start,0,0,1,0022300012,0.0,12,0,0,428,1944,673,MEM,UTA,home,0022300012,573.0,2486.0,542.0,False,0022300012,502.0,2180.0,236.0
41731,7,PT11M47.00S,1,1610612763,MEM,1628991,Jackson Jr.,J. Jackson Jr.,1,65,7,Made,1,2.0,0.0,2,h,Jackson Jr. 7' Hook Shot (2 PTS) (Gilyard 1 AST),Made Shot,Hook Shot,1,2,3,0022300012,2.0,11,47,13,428,1944,673,MEM,UTA,home,0022300012,573.0,2486.0,542.0,False,0022300012,502.0,2180.0,236.0
41732,13,PT10M47.00S,1,1610612763,MEM,1630217,Bane,D. Bane,20,7,2,Made,1,4.0,0.0,4,h,Bane 2' Driving Floating Bank Jump Shot (2 PTS),Made Shot,Driving Floating Bank Jump Shot,1,2,9,0022300012,4.0,10,47,73,428,1944,673,MEM,UTA,home,0022300012,573.0,2486.0,542.0,False,0022300012,502.0,2180.0,236.0
41733,18,PT10M17.00S,1,1610612763,MEM,1631367,Gilyard,J. Gilyard,-202,172,27,Made,1,7.0,0.0,7,h,Gilyard 27' 3PT Running Jump Shot (3 PTS) (Ban...,Made Shot,Running Jump Shot,1,3,14,0022300012,7.0,10,17,103,428,1944,673,MEM,UTA,home,0022300012,573.0,2486.0,542.0,False,0022300012,502.0,2180.0,236.0
41734,25,PT09M40.00S,1,1610612762,UTA,1628374,Markkanen,L. Markkanen,54,256,26,Made,1,7.0,3.0,10,v,Markkanen 26' 3PT Jump Shot (3 PTS) (Clarkson ...,Made Shot,Jump Shot,1,3,20,0022300012,4.0,9,40,140,428,1944,673,MEM,UTA,home,0022300012,573.0,2486.0,542.0,False,0022300012,502.0,2180.0,236.0
41735,27,PT09M17.00S,1,1610612763,MEM,1628991,Jackson Jr.,J. Jackson Jr.,29,50,6,Made,1,9.0,3.0,12,h,Jackson Jr. 6' Turnaround Jump Shot (4 PTS),Made Shot,Turnaround Jump Shot,1,2,21,0022300012,6.0,9,17,163,428,1944,673,MEM,UTA,home,0022300012,573.0,2486.0,542.0,False,0022300012,502.0,2180.0,236.0
41736,28,PT09M02.00S,1,1610612762,UTA,1630534,Agbaji,O. Agbaji,153,210,26,Made,1,9.0,6.0,15,v,Agbaji 26' 3PT Jump Shot (3 PTS) (George 1 AST),Made Shot,Jump Shot,1,3,22,0022300012,3.0,9,2,178,428,1944,673,MEM,UTA,home,0022300012,573.0,2486.0,542.0,False,0022300012,502.0,2180.0,236.0
41737,33,PT08M43.00S,1,1610612763,MEM,203935,Smart,M. Smart,0,0,0,,0,10.0,6.0,16,h,Smart Free Throw 1 of 2 (1 PTS),Free Throw,Free Throw 1 of 2,1,0,24,0022300012,4.0,8,43,197,428,1944,673,MEM,UTA,home,0022300012,573.0,2486.0,542.0,False,0022300012,502.0,2180.0,236.0
41738,34,PT08M43.00S,1,1610612763,MEM,203935,Smart,M. Smart,0,0,0,,0,11.0,6.0,17,h,Smart Free Throw 2 of 2 (2 PTS),Free Throw,Free Throw 2 of 2,1,0,25,0022300012,5.0,8,43,197,428,1944,673,MEM,UTA,home,0022300012,573.0,2486.0,542.0,False,0022300012,502.0,2180.0,236.0
41739,36,PT08M18.00S,1,1610612763,MEM,1630217,Bane,D. Bane,16,200,20,Made,1,13.0,6.0,19,h,Bane 20' Jump Shot (4 PTS),Made Shot,Jump Shot,1,2,27,0022300012,7.0,8,18,222,428,1944,673,MEM,UTA,home,0022300012,573.0,2486.0,542.0,False,0022300012,502.0,2180.0,236.0


In [36]:
all_plays_fil.drop("within_2_possessions", axis=1, inplace=True)

### Clutch Definition
5 Point game with <= 5 minutes left in game

In [37]:
all_plays_fil["clutch_definition"] = np.where(
    all_plays_fil["actionNumber"] > all_plays_fil["lead_20_actionNumber"], 
    (all_plays_fil["point_diff"].abs() <= 5)
    & (all_plays_fil["period"] < 4)
    & (all_plays_fil["minutes"] <= 5),
    False
)
all_plays_fil["clutch_definition_game_id"] = np.where(
    all_plays_fil["clutch_definition"], all_plays_fil["game_id"], None
)

In [38]:
game_ids_clutch_definition = all_plays_fil.loc[
    all_plays_fil["clutch_definition"]
].groupby("clutch_definition_game_id").agg({
    "actionNumber": "min",
    "seconds_elapsed": "min"
}).reset_index().rename(
    columns={
        "actionNumber": "clutch_definition_actionNumber",
        "seconds_elapsed": "clutch_definition_seconds_elapsed"
    }
)

In [39]:
all_plays_fil = all_plays_fil.merge(
    game_ids_clutch_definition,
    left_on="game_id",
    right_on="clutch_definition_game_id",
    how="left"
).drop("clutch_definition_game_id_x", axis=1).rename(columns={
    "clutch_definition_game_id_y": "clutch_definition_game_id"
})

In [40]:
all_plays_fil["seconds_to_clutch_definition"] = np.where(
    pd.isnull(all_plays_fil["clutch_definition_actionNumber"]),
    np.NaN,
    all_plays_fil["clutch_definition_seconds_elapsed"]-all_plays_fil["lead_20_seconds_elapsed"]
)

In [41]:
all_plays_fil.drop("clutch_definition", axis=1, inplace=True)

In [42]:
game_ids_20_point_leads.loc[game_ids_20_point_leads["game_id"]=='0022300886']

,game_id,lead_20_actionNumber,lead_20_seconds_elapsed,end_actionNumber
418,0022300886,504,2313,618


# Analysis

In [131]:
# valid_score_df = all_plays_fil.loc[~pd.isnull(all_plays_fil["scoreHome"])]
dff = all_plays_fil.loc[:,
    [
        "game_id","scoreHome", "scoreAway", "point_diff", "seconds_elapsed", "home_team", "away_team",
        "actionNumber",
        "lead_20_actionNumber", "lead_20_seconds_elapsed",
        "tied_lost_lead_game_id", "within_2_possessions_game_id", "clutch_definition_game_id",
        "tied_lost_lead_actionNumber", "tied_lost_lead_seconds_elapsed", "seconds_to_tied_lost_lead",
        "within_2_possessions_actionNumber", "within_2_possessions_seconds_elapsed", "seconds_to_within_2_possessions",
        "clutch_definition_actionNumber", "clutch_definition_seconds_elapsed", "seconds_to_clutch_definition",
        "end_actionNumber"
    ]
].merge(
    all_plays_fil.loc[
        all_plays_fil["actionNumber"] == all_plays_fil["lead_20_actionNumber"],
        ["game_id", "point_diff"]
    ].rename(columns={"point_diff": "start_point_diff"}),
    on="game_id"
).merge(
    all_plays_fil.loc[
        all_plays_fil["actionNumber"] == all_plays_fil["end_actionNumber"],
        ["game_id", "point_diff"]
    ].rename(columns={"point_diff": "end_point_diff"}),
    on="game_id"
).assign(
    start_winning_team=lambda row: np.where(row["start_point_diff"] > 0, row["home_team"], row["away_team"]),
    end_winning_team=lambda row: np.where(row["end_point_diff"] > 0, row["home_team"], row["away_team"]),
    diff_of_diffs=lambda row: (row["end_point_diff"] - row["start_point_diff"])
).assign(
    diff_of_diffs=lambda row: np.where(row["start_point_diff"] < 0, row["diff_of_diffs"]*-1, row["diff_of_diffs"]),
    start_winning_team_won=lambda row: np.where(row["start_winning_team"]==row["end_winning_team"], row["game_id"], None),
    lead_20_half=lambda row: np.where(row["lead_20_seconds_elapsed"] <= 1440, "1H", "2H"),
    lead_20_quarter=lambda row: np.where(
        row["lead_20_seconds_elapsed"] <= 720, "1Q", np.where(
            row["lead_20_seconds_elapsed"] <= 1440, "2Q", np.where(
                row["lead_20_seconds_elapsed"] <= 2160, "3Q", "4Q"
            )
        ))
)

In [44]:
dff.loc[
    ((dff["home_team"] == "BOS") & (dff["away_team"] == "CLE"))
    | ((dff["home_team"] == "CLE") & (dff["away_team"] == "BOS"))
]["game_id"].unique()

array(['0022400852', '0022400710', '0022300886', '0022400021'],
      dtype=object)

In [45]:
dff.loc[
    (dff["game_id"].isin(['0022300319', '0022400307', '0022300305', '0022400710',
       '0022400852', '0022400021', '0022300886']))
#     & ((dff["actionNumber"] == dff["end_actionNumber"]) | (dff["actionNumber"] == dff["lead_20_actionNumber"]))
].sort_values("point_diff").head(100)

,game_id,scoreHome,scoreAway,point_diff,seconds_elapsed,home_team,away_team,actionNumber,lead_20_actionNumber,lead_20_seconds_elapsed,tied_lost_lead_game_id,within_2_possessions_game_id,clutch_definition_game_id,tied_lost_lead_actionNumber,tied_lost_lead_seconds_elapsed,seconds_to_tied_lost_lead,within_2_possessions_actionNumber,within_2_possessions_seconds_elapsed,seconds_to_within_2_possessions,clutch_definition_actionNumber,clutch_definition_seconds_elapsed,seconds_to_clutch_definition,end_actionNumber,start_point_diff,end_point_diff,start_winning_team,end_winning_team,diff_of_diffs,start_winning_team_won
122906,0022300886,71.0,93.0,-22.0,2340,CLE,BOS,508,504,2313,0022300886,0022300886,NaN,575.0,2681.0,368.0,560.0,2589.0,276.0,NaN,NaN,NaN,618,-20.0,1.0,BOS,CLE,-21.0,None
20211,0022400710,15.0,35.0,-20.0,777,CLE,BOS,193,193,777,NaN,0022400710,NaN,NaN,NaN,NaN,442.0,1818.0,1041.0,NaN,NaN,NaN,699,-20.0,-7.0,BOS,BOS,-13.0,0022400710
20212,0022400710,15.0,35.0,-20.0,784,CLE,BOS,197,193,777,NaN,0022400710,NaN,NaN,NaN,NaN,442.0,1818.0,1041.0,NaN,NaN,NaN,699,-20.0,-7.0,BOS,BOS,-13.0,0022400710
122905,0022300886,71.0,91.0,-20.0,2313,CLE,BOS,504,504,2313,0022300886,0022300886,NaN,575.0,2681.0,368.0,560.0,2589.0,276.0,NaN,NaN,NaN,618,-20.0,1.0,BOS,CLE,-21.0,None
20221,0022400710,23.0,43.0,-20.0,946,CLE,BOS,234,193,777,NaN,0022400710,NaN,NaN,NaN,NaN,442.0,1818.0,1041.0,NaN,NaN,NaN,699,-20.0,-7.0,BOS,BOS,-13.0,0022400710
20218,0022400710,20.0,40.0,-20.0,884,CLE,BOS,218,193,777,NaN,0022400710,NaN,NaN,NaN,NaN,442.0,1818.0,1041.0,NaN,NaN,NaN,699,-20.0,-7.0,BOS,BOS,-13.0,0022400710
20224,0022400710,26.0,45.0,-19.0,1028,CLE,BOS,262,193,777,NaN,0022400710,NaN,NaN,NaN,NaN,442.0,1818.0,1041.0,NaN,NaN,NaN,699,-20.0,-7.0,BOS,BOS,-13.0,0022400710
122907,0022300886,74.0,93.0,-19.0,2348,CLE,BOS,510,504,2313,0022300886,0022300886,NaN,575.0,2681.0,368.0,560.0,2589.0,276.0,NaN,NaN,NaN,618,-20.0,1.0,BOS,CLE,-21.0,None
20213,0022400710,16.0,35.0,-19.0,784,CLE,BOS,199,193,777,NaN,0022400710,NaN,NaN,NaN,NaN,442.0,1818.0,1041.0,NaN,NaN,NaN,699,-20.0,-7.0,BOS,BOS,-13.0,0022400710
20210,0022400710,15.0,34.0,-19.0,777,CLE,BOS,190,193,777,NaN,0022400710,NaN,NaN,NaN,NaN,442.0,1818.0,1041.0,NaN,NaN,NaN,699,-20.0,-7.0,BOS,BOS,-13.0,0022400710


In [46]:
dff.loc[
    (dff["start_winning_team"] == "BOS")
    & (dff["end_winning_team"] == "BOS")
    & (dff["end_point_diff"].abs() > 6)
    & (dff["end_point_diff"].abs() < 10)
    & pd.isnull(dff["clutch_definition_game_id"])
]

,game_id,scoreHome,scoreAway,point_diff,seconds_elapsed,home_team,away_team,actionNumber,lead_20_actionNumber,lead_20_seconds_elapsed,tied_lost_lead_game_id,within_2_possessions_game_id,clutch_definition_game_id,tied_lost_lead_actionNumber,tied_lost_lead_seconds_elapsed,seconds_to_tied_lost_lead,within_2_possessions_actionNumber,within_2_possessions_seconds_elapsed,seconds_to_within_2_possessions,clutch_definition_actionNumber,clutch_definition_seconds_elapsed,seconds_to_clutch_definition,end_actionNumber,start_point_diff,end_point_diff,start_winning_team,end_winning_team,diff_of_diffs,start_winning_team_won
20183,0022400710,0.0,0.0,0.0,0,CLE,BOS,2,193,777,NaN,0022400710,NaN,NaN,NaN,NaN,442.0,1818.0,1041.0,NaN,NaN,NaN,699,-20.0,-7.0,BOS,BOS,-13.0,0022400710
20184,0022400710,2.0,0.0,2.0,64,CLE,BOS,17,193,777,NaN,0022400710,NaN,NaN,NaN,NaN,442.0,1818.0,1041.0,NaN,NaN,NaN,699,-20.0,-7.0,BOS,BOS,-13.0,0022400710
20185,0022400710,2.0,2.0,0.0,86,CLE,BOS,19,193,777,NaN,0022400710,NaN,NaN,NaN,NaN,442.0,1818.0,1041.0,NaN,NaN,NaN,699,-20.0,-7.0,BOS,BOS,-13.0,0022400710
20186,0022400710,3.0,2.0,1.0,99,CLE,BOS,23,193,777,NaN,0022400710,NaN,NaN,NaN,NaN,442.0,1818.0,1041.0,NaN,NaN,NaN,699,-20.0,-7.0,BOS,BOS,-13.0,0022400710
20187,0022400710,4.0,2.0,2.0,99,CLE,BOS,24,193,777,NaN,0022400710,NaN,NaN,NaN,NaN,442.0,1818.0,1041.0,NaN,NaN,NaN,699,-20.0,-7.0,BOS,BOS,-13.0,0022400710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98472,0022300771,108.0,114.0,-6.0,2753,BKN,BOS,635,331,1384,NaN,0022300771,NaN,NaN,NaN,NaN,577.0,2405.0,1021.0,NaN,NaN,NaN,653,-20.0,-8.0,BOS,BOS,-12.0,0022300771
98473,0022300771,108.0,116.0,-8.0,2772,BKN,BOS,637,331,1384,NaN,0022300771,NaN,NaN,NaN,NaN,577.0,2405.0,1021.0,NaN,NaN,NaN,653,-20.0,-8.0,BOS,BOS,-12.0,0022300771
98474,0022300771,108.0,118.0,-10.0,2834,BKN,BOS,649,331,1384,NaN,0022300771,NaN,NaN,NaN,NaN,577.0,2405.0,1021.0,NaN,NaN,NaN,653,-20.0,-8.0,BOS,BOS,-12.0,0022300771
98475,0022300771,110.0,118.0,-8.0,2841,BKN,BOS,651,331,1384,NaN,0022300771,NaN,NaN,NaN,NaN,577.0,2405.0,1021.0,NaN,NaN,NaN,653,-20.0,-8.0,BOS,BOS,-12.0,0022300771


## Win %

In [133]:
dff.loc[
    dff["actionNumber"]==dff["lead_20_actionNumber"]
].groupby(["start_winning_team", "lead_20_half"]).agg({
    "game_id": "nunique",
    "start_winning_team_won": "nunique",
}).reset_index().assign(
    win_pct=lambda row: row["start_winning_team_won"]/row["game_id"],
    losses=lambda row: row["game_id"] - row["start_winning_team_won"]
).sort_values(
    ["game_id", "win_pct"], 
    ascending=False
).reset_index().drop("index", axis=1)

,start_winning_team,lead_20_half,game_id,start_winning_team_won,win_pct,losses
0,OKC,2H,48,47,0.979167,1
1,BOS,2H,46,45,0.978261,1
2,MIN,2H,41,40,0.975610,1
3,DEN,2H,38,37,0.973684,1
4,BOS,1H,34,32,0.941176,2
5,LAC,2H,33,30,0.909091,3
6,MIL,2H,31,30,0.967742,1
7,OKC,1H,31,30,0.967742,1
8,IND,2H,30,30,1.000000,0
9,ORL,2H,30,30,1.000000,0


## Point Differential

In [134]:
dff.loc[
    dff["actionNumber"]==dff["lead_20_actionNumber"]
].groupby(["start_winning_team", "lead_20_half"]).agg({
    "game_id": "nunique",
    "start_winning_team_won": "nunique",
    "diff_of_diffs": "mean",
}).reset_index().assign(
    win_pct=lambda row: row["start_winning_team_won"]/row["game_id"],
    losses=lambda row: row["game_id"] - row["start_winning_team_won"]
).sort_values(
    "diff_of_diffs", 
    ascending=False
).reset_index().drop("index", axis=1)

,start_winning_team,lead_20_half,game_id,start_winning_team_won,diff_of_diffs,win_pct,losses
0,OKC,1H,31,30,5.967742,0.967742,1
1,SAS,2H,15,15,2.761905,1.000000,0
2,PHI,1H,11,10,2.545455,0.909091,1
3,IND,1H,9,9,2.222222,1.000000,0
4,NYK,1H,24,23,1.666667,0.958333,1
5,IND,2H,30,30,1.250000,1.000000,0
6,MEM,2H,16,16,0.812500,1.000000,0
7,SAC,1H,13,11,0.692308,0.846154,2
8,MIL,1H,20,20,0.450000,1.000000,0
9,MEM,1H,17,16,0.235294,0.941176,1


## Time-to-20

In [49]:
dff.loc[
    dff["actionNumber"]==dff["lead_20_actionNumber"]
].groupby("start_winning_team").agg({
    "game_id": "nunique",
    "start_winning_team_won": "nunique",
    "diff_of_diffs": "mean",
    "lead_20_seconds_elapsed": "mean"
}).reset_index().assign(
    win_pct=lambda row: row["start_winning_team_won"]/row["game_id"],
    losses=lambda row: row["game_id"] - row["start_winning_team_won"],
    mins_elapsed=lambda row: row["lead_20_seconds_elapsed"]/60
).sort_values(
    "lead_20_seconds_elapsed", 
    ascending=True
).reset_index().drop("index", axis=1)

,start_winning_team,game_id,start_winning_team_won,diff_of_diffs,lead_20_seconds_elapsed,win_pct,losses,mins_elapsed
0,DAL,41,39,-2.097561,1416.487805,0.951220,2,23.608130
1,DET,23,21,-4.086957,1428.130435,0.913043,2,23.802174
2,NOP,40,36,-4.625000,1429.800000,0.900000,4,23.830000
3,CLE,54,52,-0.592593,1488.907407,0.962963,2,24.815123
4,HOU,44,42,-1.568182,1507.863636,0.954545,2,25.131061
5,TOR,24,21,-5.375000,1519.041667,0.875000,3,25.317361
6,ATL,33,31,-6.030303,1572.242424,0.939394,2,26.204040
7,MEM,33,32,0.515152,1575.363636,0.969697,1,26.256061
8,LAL,45,45,-3.933333,1584.888889,1.000000,0,26.414815
9,PHX,39,37,-4.743590,1587.000000,0.948718,2,26.450000


## Blown Leads

### Tied or Lost Lead

In [50]:
pprint(list(dff.columns))

['game_id',
 'scoreHome',
 'scoreAway',
 'point_diff',
 'seconds_elapsed',
 'home_team',
 'away_team',
 'actionNumber',
 'lead_20_actionNumber',
 'lead_20_seconds_elapsed',
 'tied_lost_lead_game_id',
 'within_2_possessions_game_id',
 'clutch_definition_game_id',
 'tied_lost_lead_actionNumber',
 'tied_lost_lead_seconds_elapsed',
 'seconds_to_tied_lost_lead',
 'within_2_possessions_actionNumber',
 'within_2_possessions_seconds_elapsed',
 'seconds_to_within_2_possessions',
 'clutch_definition_actionNumber',
 'clutch_definition_seconds_elapsed',
 'seconds_to_clutch_definition',
 'end_actionNumber',
 'start_point_diff',
 'end_point_diff',
 'start_winning_team',
 'end_winning_team',
 'diff_of_diffs',
 'start_winning_team_won']


In [51]:
dff.groupby("start_winning_team").agg({
    "game_id": "nunique",
    "start_winning_team_won": "nunique",
    "diff_of_diffs": "mean",
    "lead_20_seconds_elapsed": "mean",
    "tied_lost_lead_game_id": "nunique",
    "seconds_to_tied_lost_lead": "mean"
}).reset_index().assign(
    win_pct=lambda row: row["start_winning_team_won"]/row["game_id"],
    losses=lambda row: row["game_id"] - row["start_winning_team_won"],
    mins_elapsed=lambda row: row["lead_20_seconds_elapsed"]/60,
    mins_to_tied_lost_lead=lambda row: row["seconds_to_tied_lost_lead"]/60,
    tied_lost_lead_pct=lambda row: row["tied_lost_lead_game_id"]/row["game_id"]
).sort_values(
    "tied_lost_lead_pct", 
    ascending=True
).reset_index()

,index,start_winning_team,game_id,start_winning_team_won,diff_of_diffs,lead_20_seconds_elapsed,tied_lost_lead_game_id,seconds_to_tied_lost_lead,win_pct,losses,mins_elapsed,mins_to_tied_lost_lead,tied_lost_lead_pct
0,19,NYK,53,52,-0.278592,1672.502644,1,1623.000000,0.981132,1,27.875044,27.050000,0.018868
1,13,LAL,45,45,-4.097565,1579.777984,1,1366.000000,1.000000,0,26.329633,22.766667,0.022222
2,22,PHI,27,25,-1.605586,1673.952693,1,1107.000000,0.925926,2,27.899212,18.450000,0.037037
3,7,DEN,50,49,-1.817331,1692.115036,2,1101.052632,0.980000,1,28.201917,18.350877,0.040000
4,10,HOU,44,42,-1.706300,1496.251990,2,1272.643791,0.954545,2,24.937533,21.210730,0.045455
5,21,ORL,40,38,-3.154019,1720.449816,2,1151.500000,0.950000,2,28.674164,19.191667,0.050000
6,23,PHX,39,37,-4.810065,1568.965097,2,1106.351351,0.948718,2,26.149418,18.439189,0.051282
7,11,IND,39,39,1.576305,1884.665830,2,956.712230,1.000000,0,31.411097,15.945204,0.051282
8,5,CLE,54,52,-0.866178,1487.152311,3,1571.329897,0.962963,2,24.785872,26.188832,0.055556
9,14,MEM,33,32,0.602807,1583.293567,2,1087.110672,0.969697,1,26.388226,18.118511,0.060606


### Within 2 Possessions

In [52]:
dff.groupby("start_winning_team").agg({
    "game_id": "nunique",
    "start_winning_team_won": "nunique",
    "diff_of_diffs": "mean",
    "within_2_possessions_seconds_elapsed": "mean",
    "within_2_possessions_game_id": "nunique",
    "seconds_to_within_2_possessions": "mean",
    "lead_20_seconds_elapsed": "mean",
    "tied_lost_lead_game_id": "nunique",
    "seconds_to_tied_lost_lead": "mean"
}).reset_index().assign(
    win_pct=lambda row: row["start_winning_team_won"]/row["game_id"],
    losses=lambda row: row["game_id"] - row["start_winning_team_won"],
    mins_elapsed=lambda row: row["lead_20_seconds_elapsed"]/60,
    mins_to_within_2_possessions=lambda row: row["seconds_to_within_2_possessions"]/60,
    within_2_possessions_pct=lambda row: row["within_2_possessions_game_id"]/row["game_id"],
    mins_to_tied_lost_lead=lambda row: row["seconds_to_tied_lost_lead"]/60,
    tied_lost_lead_pct=lambda row: row["tied_lost_lead_game_id"]/row["game_id"]
).sort_values(
    "within_2_possessions_pct", 
    ascending=True
)

,start_winning_team,game_id,start_winning_team_won,diff_of_diffs,within_2_possessions_seconds_elapsed,within_2_possessions_game_id,seconds_to_within_2_possessions,lead_20_seconds_elapsed,tied_lost_lead_game_id,seconds_to_tied_lost_lead,win_pct,losses,mins_elapsed,mins_to_within_2_possessions,within_2_possessions_pct,mins_to_tied_lost_lead,tied_lost_lead_pct
14,MEM,33,32,0.602807,1830.312373,4,854.768763,1583.293567,2,1087.110672,0.969697,1,26.388226,14.246146,0.121212,18.118511,0.060606
5,CLE,54,52,-0.866178,2173.743079,7,1176.524917,1487.152311,3,1571.329897,0.962963,2,24.785872,19.608749,0.129630,26.188832,0.055556
12,LAC,54,49,-2.617885,1930.936455,7,539.685619,1641.016758,6,836.292994,0.907407,5,27.350279,8.994760,0.129630,13.938217,0.111111
17,MIN,53,50,-1.767765,2124.736178,8,696.074685,1880.317170,5,1040.758621,0.943396,3,31.338619,11.601245,0.150943,17.345977,0.094340
19,NYK,53,52,-0.278592,2213.370306,9,858.431441,1672.502644,1,1623.000000,0.981132,1,27.875044,14.307191,0.169811,27.050000,0.018868
21,ORL,40,38,-3.154019,2085.906542,7,723.511682,1720.449816,2,1151.500000,0.950000,2,28.674164,12.058528,0.175000,19.191667,0.050000
20,OKC,79,77,0.926545,1983.286830,14,759.616071,1707.059021,5,1122.329866,0.974684,2,28.450984,12.660268,0.177215,18.705498,0.063291
7,DEN,50,49,-1.817331,2283.952498,9,812.361998,1692.115036,2,1101.052632,0.980000,1,28.201917,13.539367,0.180000,18.350877,0.040000
22,PHI,27,25,-1.605586,2337.624060,5,770.485714,1673.952693,1,1107.000000,0.925926,2,27.899212,12.841429,0.185185,18.450000,0.037037
10,HOU,44,42,-1.706300,1891.758649,9,757.803954,1496.251990,2,1272.643791,0.954545,2,24.937533,12.630066,0.204545,21.210730,0.045455


### Clutch Definition

In [53]:
dff.groupby("start_winning_team").agg({
    "game_id": "nunique",
    "start_winning_team_won": "nunique",
    "diff_of_diffs": "mean",
    "clutch_definition_seconds_elapsed": "mean",
    "clutch_definition_game_id": "nunique",
    "seconds_to_clutch_definition": "mean",
    "within_2_possessions_seconds_elapsed": "mean",
    "within_2_possessions_game_id": "nunique",
    "seconds_to_within_2_possessions": "mean",
    "lead_20_seconds_elapsed": "mean",
    "tied_lost_lead_game_id": "nunique",
    "seconds_to_tied_lost_lead": "mean"
}).reset_index().assign(
    win_pct=lambda row: row["start_winning_team_won"]/row["game_id"],
    losses=lambda row: row["game_id"] - row["start_winning_team_won"],
    mins_elapsed=lambda row: row["lead_20_seconds_elapsed"]/60,
    mins_to_clutch_definition=lambda row: row["seconds_to_within_2_possessions"]/60,
    clutch_definition_pct=lambda row: row["clutch_definition_game_id"]/row["game_id"],
    mins_to_within_2_possessions=lambda row: row["seconds_to_within_2_possessions"]/60,
    within_2_possessions_pct=lambda row: row["within_2_possessions_game_id"]/row["game_id"],
    mins_to_tied_lost_lead=lambda row: row["seconds_to_tied_lost_lead"]/60,
    tied_lost_lead_pct=lambda row: row["tied_lost_lead_game_id"]/row["game_id"]
).sort_values(
    "clutch_definition_pct", 
    ascending=True
)

,start_winning_team,game_id,start_winning_team_won,diff_of_diffs,clutch_definition_seconds_elapsed,clutch_definition_game_id,seconds_to_clutch_definition,within_2_possessions_seconds_elapsed,within_2_possessions_game_id,seconds_to_within_2_possessions,lead_20_seconds_elapsed,tied_lost_lead_game_id,seconds_to_tied_lost_lead,win_pct,losses,mins_elapsed,mins_to_clutch_definition,clutch_definition_pct,mins_to_within_2_possessions,within_2_possessions_pct,mins_to_tied_lost_lead,tied_lost_lead_pct
5,CLE,54,52,-0.866178,2152.000000,1,1491.000000,2173.743079,7,1176.524917,1487.152311,3,1571.329897,0.962963,2,24.785872,19.608749,0.018519,19.608749,0.129630,26.188832,0.055556
13,LAL,45,45,-4.097565,1825.000000,1,940.000000,2162.011843,10,997.450777,1579.777984,1,1366.000000,1.000000,0,26.329633,16.624180,0.022222,16.624180,0.222222,22.766667,0.022222
22,PHI,27,25,-1.605586,2047.000000,1,1031.000000,2337.624060,5,770.485714,1673.952693,1,1107.000000,0.925926,2,27.899212,12.841429,0.037037,12.841429,0.185185,18.450000,0.037037
7,DEN,50,49,-1.817331,1931.000000,2,1215.197802,2283.952498,9,812.361998,1692.115036,2,1101.052632,0.980000,1,28.201917,13.539367,0.040000,13.539367,0.180000,18.350877,0.040000
11,IND,39,39,1.576305,1887.945652,2,647.891304,2119.811676,8,665.258945,1884.665830,2,956.712230,1.000000,0,31.411097,11.087649,0.051282,11.087649,0.205128,15.945204,0.051282
23,PHX,39,37,-4.810065,1818.800000,2,1134.753846,2320.217850,8,1080.599808,1568.965097,2,1106.351351,0.948718,2,26.149418,18.009997,0.051282,18.009997,0.205128,18.439189,0.051282
19,NYK,53,52,-0.278592,1958.277174,3,910.584239,2213.370306,9,858.431441,1672.502644,1,1623.000000,0.981132,1,27.875044,14.307191,0.056604,14.307191,0.169811,27.050000,0.018868
17,MIN,53,50,-1.767765,1722.565327,3,632.045226,2124.736178,8,696.074685,1880.317170,5,1040.758621,0.943396,3,31.338619,11.601245,0.056604,11.601245,0.150943,17.345977,0.094340
25,SAC,43,39,-3.174471,1818.557252,3,653.819338,2012.404239,10,726.141559,1718.363045,5,1055.410575,0.906977,4,28.639384,12.102359,0.069767,12.102359,0.232558,17.590176,0.116279
12,LAC,54,49,-2.617885,1610.928983,4,599.604607,1930.936455,7,539.685619,1641.016758,6,836.292994,0.907407,5,27.350279,8.994760,0.074074,8.994760,0.129630,13.938217,0.111111


In [137]:
dff.groupby(["start_winning_team", "lead_20_half"]).agg({
    "game_id": "nunique",
    "start_winning_team_won": "nunique",
    "diff_of_diffs": "mean",
    "clutch_definition_seconds_elapsed": "mean",
    "clutch_definition_game_id": "nunique",
    "seconds_to_clutch_definition": "mean",
    "within_2_possessions_seconds_elapsed": "mean",
    "within_2_possessions_game_id": "nunique",
    "seconds_to_within_2_possessions": "mean",
    "lead_20_seconds_elapsed": "mean",
    "tied_lost_lead_game_id": "nunique",
    "seconds_to_tied_lost_lead": "mean"
}).reset_index().assign(
    win_pct=lambda row: row["start_winning_team_won"]/row["game_id"],
    losses=lambda row: row["game_id"] - row["start_winning_team_won"],
    mins_elapsed=lambda row: row["lead_20_seconds_elapsed"]/60,
    mins_to_clutch_definition=lambda row: row["seconds_to_within_2_possessions"]/60,
    clutch_definition_pct=lambda row: row["clutch_definition_game_id"]/row["game_id"],
    mins_to_within_2_possessions=lambda row: row["seconds_to_within_2_possessions"]/60,
    within_2_possessions_pct=lambda row: row["within_2_possessions_game_id"]/row["game_id"],
    mins_to_tied_lost_lead=lambda row: row["seconds_to_tied_lost_lead"]/60,
    tied_lost_lead_pct=lambda row: row["tied_lost_lead_game_id"]/row["game_id"]
).sort_values(
    "within_2_possessions_pct", 
    ascending=True
)

,start_winning_team,lead_20_half,game_id,start_winning_team_won,diff_of_diffs,clutch_definition_seconds_elapsed,clutch_definition_game_id,seconds_to_clutch_definition,within_2_possessions_seconds_elapsed,within_2_possessions_game_id,seconds_to_within_2_possessions,lead_20_seconds_elapsed,tied_lost_lead_game_id,seconds_to_tied_lost_lead,win_pct,losses,mins_elapsed,mins_to_clutch_definition,clutch_definition_pct,mins_to_within_2_possessions,within_2_possessions_pct,mins_to_tied_lost_lead,tied_lost_lead_pct
29,MEM,2H,16,16,0.899184,NaN,0,NaN,NaN,0,NaN,2096.156025,0,NaN,1.000000,0,34.935934,NaN,0.000000,NaN,0.000000,NaN,0.000000
11,CLE,2H,26,26,-1.155162,NaN,0,NaN,NaN,0,NaN,2045.200061,0,NaN,1.000000,0,34.086668,NaN,0.000000,NaN,0.000000,NaN,0.000000
21,HOU,2H,19,19,0.334410,NaN,0,NaN,2311.000000,1,387.000000,2044.223748,0,NaN,1.000000,0,34.070396,6.450000,0.000000,6.450000,0.052632,NaN,0.000000
57,UTA,2H,14,14,-3.667219,NaN,0,NaN,2180.000000,1,236.000000,2073.080022,1,542.000000,1.000000,0,34.551334,3.933333,0.000000,3.933333,0.071429,9.033333,0.071429
39,NYK,2H,29,29,-1.815681,NaN,0,NaN,2407.791262,3,556.592233,2187.732325,0,NaN,1.000000,0,36.462205,9.276537,0.000000,9.276537,0.103448,NaN,0.000000
1,ATL,2H,19,19,-3.340964,1863.000000,1,260.000000,2223.349593,2,532.069106,1967.691165,1,589.000000,1.000000,0,32.794853,8.867818,0.052632,8.867818,0.105263,9.816667,0.052632
13,DAL,2H,18,18,-3.005249,NaN,0,NaN,2550.904762,2,778.345238,1962.093176,1,668.000000,1.000000,0,32.701553,12.972421,0.000000,12.972421,0.111111,11.133333,0.055556
37,NOP,2H,17,17,-3.124576,NaN,0,NaN,2383.497738,2,768.791855,2181.471159,0,NaN,1.000000,0,36.357853,12.813198,0.000000,12.813198,0.117647,NaN,0.000000
25,LAC,2H,33,30,-3.677056,1834.000000,1,380.000000,2304.634051,4,509.444227,2043.293883,3,704.270677,0.909091,3,34.054898,8.490737,0.030303,8.490737,0.121212,11.737845,0.090909
35,MIN,2H,41,40,-1.245562,1991.000000,1,486.000000,2422.466346,5,731.881410,2151.936292,2,686.923077,0.975610,1,35.865605,12.198024,0.024390,12.198024,0.121951,11.448718,0.048780


## Shot Profile

In [54]:
all_plays["actionType"].unique()

array(['period', 'Jump Ball', 'Made Shot', 'Turnover', '', 'Foul',
       'Free Throw', 'Rebound', 'Missed Shot', 'Timeout', 'Substitution',
       'Instant Replay', 'Violation', 'Ejection'], dtype=object)

In [138]:
shot_profile = all_plays.loc[
    (all_plays["isFieldGoal"]==1) | (all_plays["actionType"].isin(["Free Throw", "Turnover"]))
].merge(
    game_ids_20_point_leads,
    on="game_id"
).merge(
    games[["GAME_ID", "home_team", "away_team"]].drop_duplicates(),
    left_on="game_id",
    right_on="GAME_ID"
).assign(
    leading_team=lambda row: np.where(row["point_diff"]<0, "away", "home"),
    lead_20_half=lambda row: np.where(row["lead_20_seconds_elapsed"] <= 1440, "1H", "2H"),
    lead_20_quarter=lambda row: np.where(
        row["lead_20_seconds_elapsed"] <= 720, "1Q", np.where(
            row["lead_20_seconds_elapsed"] <= 1440, "2Q", np.where(
                row["lead_20_seconds_elapsed"] <= 2160, "3Q", "4Q"
            )
        ))
).drop("GAME_ID", axis=1).copy()

In [139]:
shot_profile["lead_20_category"] = np.where(
    shot_profile["actionNumber"] < shot_profile["lead_20_actionNumber"], 'before', 'after'
)
shot_profile["shotResult_int"] = np.where(
    shot_profile["shotResult"] == 'Made', 1, 0
)

### FG% and Rate

In [140]:
shot_profile.groupby(
    ["teamTricode", "lead_20_category", "shotValue"]
).agg({
    "shotResult_int": ["sum", "count"]
}).reset_index()

teamTricode lead_20_category shotValue shotResult_int      
                                                      sum count
0                          after         0              0  1109
1                         before         0              0   761
2           ATL            after         0              0  1296
3           ATL            after         2            951  1691
4           ATL            after         3            460  1237
5           ATL           before         0              0  1332
6           ATL           before         2           1225  2274
7           ATL           before         3            568  1596
8           BKN            after         0              0  1217
9           BKN            after         2            795  1520
10          BKN            after         3            421  1204
11          BKN           before         0              0  1435
12          BKN           before         2           1211  2393
13          BKN           before         3            599  1722
14          BOS            after         0              0  1296
15          BOS            after         2           1050  1875
16          BOS            after         3            666  1786
17          BOS           before         0              0  1408
18          BOS           before         2           1350  2315
19          BOS           before         3            997  2351
20          CHA            after         0              0  1251
21          CHA            after         2            928  1729
22          CHA            after         3            455  1272
23          CHA           before         0              0  1334
24          CHA           before         2           1070  2258
25          CHA           before         3            481  1535
26          CHI            after         0              0  1114
27          CHI            after         2            898  1571
28          CHI            after         3            387  1075
29          CHI           before         0              0  1191
30          CHI           before         2           1097  2019
31          CHI           before         3            513  1467
32          CLE            after         0              0  1262
33          CLE            after         2            994  1719
34          CLE            after         3            551  1449
35          CLE           before         0              0  1163
36          CLE           before         2           1145  1971
37          CLE           before         3            626  1582
38          DAL            after         0              0  1334
39          DAL            after         2           1046  1853
40          DAL            after         3            486  1358
41          DAL           before         0              0  1416
42          DAL           before         2           1287  2326
43          DAL           before         3            594  1620
44          DEN            after         0              0  1118
45          DEN            after         2           1043  1787
46          DEN            after         3            412  1101
47          DEN           before         0              0  1564
48          DEN           before         2           1519  2621
49          DEN           before         3            581  1453
50          DET            after         0              0  1317
51          DET            after         2           1007  1812
52          DET            after         3            427  1219
53          DET           before         0              0  1437
54          DET           before         2           1202  2326
55          DET           before         3            471  1379
56          GSW            after         0              0  1007
57          GSW            after         2            774  1365
58          GSW            after         3            395  1056
59          GSW           before         0              0  1019
60          GSW           before    

In [147]:
shot_profile_pivot = shot_profile.pivot_table(
    index="teamTricode",
    columns=["lead_20_category", "shotValue", "lead_20_half"],
    values="shotResult_int",
    aggfunc=["sum", "count"]
).reset_index()

In [148]:
shot_profile_pivot

teamTricode   sum                                          \
lead_20_category             after                                  before   
shotValue                        0           2             3             0   
lead_20_half                    1H   2H     1H     2H     1H     2H     1H   
0                              0.0  0.0    NaN    NaN    NaN    NaN    0.0   
1                        ATL   0.0  0.0  580.0  371.0  278.0  182.0    0.0   
2                        BKN   0.0  0.0  481.0  314.0  252.0  169.0    0.0   
3                        BOS   0.0  0.0  636.0  414.0  401.0  265.0    0.0   
4                        CHA   0.0  0.0  589.0  339.0  280.0  175.0    0.0   
5                        CHI   0.0  0.0  507.0  391.0  222.0  165.0    0.0   
6                        CLE   0.0  0.0  647.0  347.0  364.0  187.0    0.0   
7                        DAL   0.0  0.0  663.0  383.0  302.0  184.0    0.0   
8                        DEN   0.0  0.0  526.0  517.0  182.0  230.0    0.0   
9                        DET   0.0  0.0  641.0  366.0  276.0  151.0    0.0   
10                       GSW   0.0  0.0  438.0  336.0  218.0  177.0    0.0   
11                       HOU   0.0  0.0  717.0  291.0  355.0  113.0    0.0   
12                       IND   0.0  0.0  530.0  472.0  207.0  168.0    0.0   
13                       LAC   0.0  0.0  677.0  385.0  289.0  154.0    0.0   
14                       LAL   0.0  0.0  608.0  351.0  270.0  191.0    0.0   
15                       MEM   0.0  0.0  582.0  400.0  283.0  207.0    0.0   
16                       MIA   0.0  0.0  272.0  389.0  135.0  216.0    0.0   
17                       MIL   0.0  0.0  562.0  351.0  299.0  197.0    0.0   
18                       MIN   0.0  0.0  423.0  389.0  198.0  184.0    0.0   
19                       NOP   0.0  0.0  874.0  390.0  368.0  160.0    0.0   
20                       NYK   0.0  0.0  640.0  312.0  336.0  149.0    0.0   
21                       OKC   0.0  0.0  770.0  400.0  423.0  194.0    0.0   
22                       ORL   0.0  0.0  406.0  386.0  169.0  196.0    0.0   
23                       PHI   0.0  0.0  445.0  342.0  217.0  142.0    0.0   
24                       PHX   0.0  0.0  474.0  379.0  259.0  212.0    0.0   
25                       POR   0.0  0.0  431.0  401.0  189.0  164.0    0.0   
26                       SAC   0.0  0.0  616.0  416.0  304.0  186.0    0.0   
27                       SAS   0.0  0.0  798.0  418.0  327.0  187.0    0.0   
28                       TOR   0.0  0.0  738.0  410.0  286.0  180.0    0.0   
29                       UTA   0.0  0.0  617.0  361.0  273.0  191.0    0.0   
30                       WAS   0.0  0.0  638.0  487.0  281.0  191.0    0.0   

                                                     count                 \
lead_20_category                                     after                  
shotValue                  2              3              0              2   
lead_20_half       2H     1H      2H     1H     2H      1H     2H      1H   
0                 0.0    NaN     NaN    NaN    NaN   542.0  567.0     NaN   
1                 0.0  318.0   907.0  161.0  407.0   779.0  517.0  1039.0   
2                 0.0  219.0   992.0  119.0  480.0   723.0  494.0   925.0   
3                 0.0  381.0   969.0  304.0  693.0   810.0  486.0  1156.0   
4                 0.0  265.0   805.0  114.0  367.0   818.0  433.0  1086.0   
5                 0.0  211.0   886.0   98.0  415.0   625.0  489.0   893.0   
6                 0.0  337.0   808.0  203.0  423.0   827.0  435.0  1147.0   
7                 0.0  367.0   920.0  192.0  402.0   843.0  491.0  1165.0   
8                 0.0  276.0  1243.0  113.0  468.0   507.0  611.0   882.0   
9                 0.0  350.0   852.0  118.0  353.0   865.0  452.0  1161.0   
10                0.0  176.0   780.0  120.0  407.0   579.0  428.0   776.0   
11                0.0  458.0   668.0  186.0  296.0   863.0  383.0  1314.0   
12                0.0  243.0  1177.0  113.0  482.0   5

In [59]:
shot_profile_pivot.columns = [
    "team",
    "after_make_0",
    "after_make_2",
    "after_make_3",
    "before_make_0",
    "before_make_2",
    "before_make_3",
    "after_att_0",
    "after_att_2",
    "after_att_3",
    "before_att_0",
    "before_att_2",
    "before_att_3",
]

In [60]:
shot_profile_pivot["after_2fg_pct"] = shot_profile_pivot["after_make_2"]/shot_profile_pivot["after_att_2"]
shot_profile_pivot["after_3fg_pct"] = shot_profile_pivot["after_make_3"]/shot_profile_pivot["after_att_3"]
shot_profile_pivot["before_2fg_pct"] = shot_profile_pivot["before_make_2"]/shot_profile_pivot["before_att_2"]
shot_profile_pivot["before_3fg_pct"] = shot_profile_pivot["before_make_3"]/shot_profile_pivot["before_att_3"]
shot_profile_pivot["after_2fg_rate"] = shot_profile_pivot["after_att_2"]/(shot_profile_pivot["after_att_2"]+shot_profile_pivot["after_att_3"])
shot_profile_pivot["after_3fg_rate"] = shot_profile_pivot["after_att_3"]/(shot_profile_pivot["after_att_2"]+shot_profile_pivot["after_att_3"])
shot_profile_pivot["before_2fg_rate"] = shot_profile_pivot["before_att_2"]/(shot_profile_pivot["before_att_2"]+shot_profile_pivot["before_att_3"])
shot_profile_pivot["before_3fg_rate"] = shot_profile_pivot["before_att_3"]/(shot_profile_pivot["before_att_2"]+shot_profile_pivot["before_att_3"])
shot_profile_pivot["diff_2fg_pct"] = shot_profile_pivot["after_2fg_pct"]-shot_profile_pivot["before_2fg_pct"]
shot_profile_pivot["diff_3fg_pct"] = shot_profile_pivot["after_3fg_pct"]-shot_profile_pivot["before_3fg_pct"]
shot_profile_pivot["diff_2fg_rate"] = shot_profile_pivot["after_2fg_rate"]-shot_profile_pivot["before_2fg_rate"]
shot_profile_pivot["diff_3fg_rate"] = shot_profile_pivot["after_3fg_rate"]-shot_profile_pivot["before_3fg_rate"]


In [61]:
shot_profile_pivot = shot_profile_pivot.loc[shot_profile_pivot["team"]!=""]

In [62]:
shot_profile_pivot.sort_values("diff_2fg_pct")

,team,after_make_0,after_make_2,after_make_3,before_make_0,before_make_2,before_make_3,after_att_0,after_att_2,after_att_3,before_att_0,before_att_2,before_att_3,after_2fg_pct,after_3fg_pct,before_2fg_pct,before_3fg_pct,after_2fg_rate,after_3fg_rate,before_2fg_rate,before_3fg_rate,diff_2fg_pct,diff_3fg_pct,diff_2fg_rate,diff_3fg_rate
3,BOS,0.0,1050.0,666.0,0.0,1350.0,997.0,1296.0,1875.0,1786.0,1408.0,2315.0,2351.0,0.560000,0.372900,0.583153,0.424075,0.512155,0.487845,0.496142,0.503858,-0.023153,-0.051175,0.016013,-0.016013
14,LAL,0.0,959.0,461.0,0.0,1475.0,634.0,1441.0,1695.0,1230.0,1673.0,2505.0,1634.0,0.565782,0.374797,0.588822,0.388005,0.579487,0.420513,0.605219,0.394781,-0.023041,-0.013208,-0.025731,0.025731
26,SAC,0.0,1032.0,490.0,0.0,1375.0,671.0,1296.0,1850.0,1338.0,1375.0,2406.0,1787.0,0.557838,0.366218,0.571488,0.375490,0.580301,0.419699,0.573813,0.426187,-0.013650,-0.009271,0.006488,-0.006488
21,OKC,0.0,1170.0,617.0,0.0,1769.0,810.0,1261.0,2060.0,1616.0,1666.0,3047.0,2044.0,0.567961,0.381807,0.580571,0.396282,0.560392,0.439608,0.598507,0.401493,-0.012610,-0.014475,-0.038115,0.038115
24,PHX,0.0,853.0,471.0,0.0,1185.0,560.0,1218.0,1549.0,1177.0,1352.0,2107.0,1455.0,0.550678,0.400170,0.562411,0.384880,0.568232,0.431768,0.591522,0.408478,-0.011733,0.015290,-0.023290,0.023290
17,MIL,0.0,913.0,496.0,0.0,1279.0,694.0,1370.0,1623.0,1326.0,1452.0,2238.0,1688.0,0.562539,0.374057,0.571492,0.411137,0.550356,0.449644,0.570046,0.429954,-0.008954,-0.037080,-0.019690,0.019690
13,LAC,0.0,1062.0,443.0,0.0,1402.0,594.0,1355.0,1854.0,1207.0,1460.0,2429.0,1517.0,0.572816,0.367026,0.577192,0.391562,0.605684,0.394316,0.615560,0.384440,-0.004377,-0.024537,-0.009876,0.009876
6,CLE,0.0,994.0,551.0,0.0,1145.0,626.0,1262.0,1719.0,1449.0,1163.0,1971.0,1582.0,0.578243,0.380262,0.580923,0.395702,0.542614,0.457386,0.554742,0.445258,-0.002680,-0.015439,-0.012129,0.012129
25,POR,0.0,832.0,353.0,0.0,1207.0,482.0,1230.0,1634.0,1080.0,1406.0,2367.0,1511.0,0.509180,0.326852,0.509928,0.318994,0.602063,0.397937,0.610366,0.389634,-0.000748,0.007858,-0.008303,0.008303
8,DEN,0.0,1043.0,412.0,0.0,1519.0,581.0,1118.0,1787.0,1101.0,1564.0,2621.0,1453.0,0.583660,0.374205,0.579550,0.399862,0.618767,0.381233,0.643348,0.356652,0.004110,-0.025657,-0.024581,0.024581


In [63]:
def dumbbell_plot(df, cat, start_metric, end_metric, start_name, end_name, axis_name, title, axis_range=None):
    teams = df[cat].unique()
    # data is a dictionary with two keys. Each key's value is a list
    data = {"line_x": [], "line_y": []}

    # Loop over unique matchups
    for t in teams:
        # Append values to end of list
        data["line_x"].extend(
            [
                df.loc[df[cat]
                                       == t][start_metric].values[0],
                df.loc[df[cat]
                                       == t][end_metric].values[0],
                None,
            ]
        )
        data["line_y"].extend([t, t, None])

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            # Line between dots
            x=data["line_x"],
            y=data["line_y"],
            mode="markers+lines",
            showlegend=False,
            marker=dict(
                symbol="arrow",
                color="black",
                size=16,
                # This makes the arrow point the correct direction
                angleref="previous",
                # This shortens the length of the arrow so it points to the dot
                #   and not into the dot
                standoff=7
            )
        ),
    )
    fig.add_trace(
        go.Scatter(
            x=df[end_metric],
            y=df[cat],
            mode="markers",
            name=end_name,
            marker=dict(
                color="orange",
                size=10
            )

        ),
    )
    fig.add_trace(
        go.Scatter(
            x=df[start_metric],
            y=df[cat],
            mode="markers",
            name=start_name,
            marker=dict(
                color="purple",
                size=10
            )
        )
    )

    fig.update_layout(
        xaxis=dict(
            title=dict(text=axis_name, font=dict(size=14)),
            gridwidth=1,
            gridcolor='lightgray',
            griddash="dot",
            range=axis_range,

        ),
        yaxis=dict(
            gridwidth=1,
            gridcolor='lightgray',
            griddash="dot",
            categoryorder='array',
            categoryarray=df.sort_values(end_metric, ascending=True)[cat].values
        ),
        plot_bgcolor="white",
        height=800,
        title=title
    )
    return fig

In [64]:
fig=dumbbell_plot(
    shot_profile_pivot, 
    "team", "before_3fg_rate", "after_3fg_rate", 
    "3FG Rate - Before", "3FG Rate - After", "3FG Rate", 
    "3FG Rate - Before v. After 20-point lead", axis_range=[0.34, 0.52]
)
fig.add_shape(
    type="rect",
    xref="paper", x0=-0.05, x1=1,
    yref="paper", y0=0.92, y1=0.95,
    line=dict(color="green"),
    opacity=0.5
)

In [65]:
fig = dumbbell_plot(
    shot_profile_pivot, 
    "team", "before_3fg_pct", "after_3fg_pct", 
    "3FG% - Before", "3FG% - After", "3FG%", 
    "3FG% - Before v. After 20-point lead", axis_range=[0.3, 0.44]
)
fig.add_shape(
    type="rect",
    xref="paper", x0=-0.05, x1=1,
    yref="paper", y0=0.62, y1=0.65,
    line=dict(color="green"),
    opacity=0.5
)

In [66]:
fig = dumbbell_plot(
    shot_profile_pivot, 
    "team", "before_2fg_pct", "after_2fg_pct", 
    "2FG% - Before", "2FG% - After", "2FG%", 
    "2FG% - Before v. After 20-point lead", axis_range=[0.46, 0.6]
)
fig.add_shape(
    type="rect",
    xref="paper", x0=-0.05, x1=1,
    yref="paper", y0=0.53, y1=0.56,
    line=dict(color="green"),
    opacity=0.5
)

### Shot Distance

In [67]:
distance = shot_profile.groupby(
    ["teamTricode", "lead_20_category", "shotValue"]
).agg({
    "shotDistance": ["mean", 'std']
}).reset_index()

In [68]:
shot_profile_pivot.columns = [
    "team",
    "after_make_0",
    "after_make_2",
    "after_make_3",
    "before_make_0",
    "before_make_2",
    "before_make_3",
    "after_att_0",
    "after_att_2",
    "after_att_3",
    "before_att_0",
    "before_att_2",
    "before_att_3",
]

ValueError: Length mismatch: Expected axis has 25 elements, new values have 13 elements

In [69]:
shot_profile

,actionNumber,clock,period,teamId,teamTricode,personId,playerName,playerNameI,xLegacy,yLegacy,shotDistance,shotResult,isFieldGoal,scoreHome,scoreAway,pointsTotal,location,description,actionType,subType,videoAvailable,shotValue,actionId,game_id,point_diff,minutes,seconds,seconds_elapsed,lead_20_actionNumber,lead_20_seconds_elapsed,end_actionNumber,home_team,away_team,leading_team,lead_20_category,shotResult_int
0,7,PT11M43.00S,1,1610612738,BOS,201950,Holiday,J. Holiday,43,7,4,Made,1,2.0,0.0,2,h,Holiday 4' Finger Roll Layup (2 PTS),Made Shot,Finger Roll Layup Shot,1,2,3,0022300118,2.0,11,43,17,210,956,646,BOS,IND,home,before,1
1,9,PT11M33.00S,1,1610612754,IND,1629614,Nembhard,A. Nembhard,-64,151,16,Made,1,2.0,2.0,4,v,Nembhard 16' Pullup Jump Shot (2 PTS),Made Shot,Pullup Jump shot,1,2,4,0022300118,0.0,11,33,27,210,956,646,BOS,IND,home,before,1
2,10,PT11M23.00S,1,1610612738,BOS,1628401,White,D. White,15,110,11,Made,1,4.0,2.0,6,h,White 11' Driving Floating Jump Shot (2 PTS),Made Shot,Driving Floating Jump Shot,1,2,5,0022300118,2.0,11,23,37,210,956,646,BOS,IND,home,before,1
3,11,PT11M00.00S,1,1610612754,IND,1626167,Turner,M. Turner,34,48,6,Made,1,4.0,4.0,8,v,Turner 6' Layup (2 PTS) (Brown 1 AST),Made Shot,Layup Shot,1,2,6,0022300118,0.0,11,0,60,210,956,646,BOS,IND,home,before,1
4,13,PT10M50.00S,1,1610612738,BOS,204001,Porziņģis,K. Porziņģis,0,0,0,,0,NaN,NaN,0,h,Porzingis Bad Pass Turnover (P1.T1),Turnover,Bad Pass,1,0,7,0022300118,NaN,10,50,70,210,956,646,BOS,IND,home,before,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292224,642,PT00M12.20S,4,1610612753,ORL,1628371,Isaac,J. Isaac,0,0,0,,0,97.0,101.0,198,v,Isaac Free Throw 2 of 2 (18 PTS),Free Throw,Free Throw 2 of 2,1,0,463,0022300033,-4.0,0,12,2868,351,1554,653,CHI,ORL,away,after,0
292225,644,PT00M07.00S,4,1610612741,CHI,203897,LaVine,Z. LaVine,162,197,26,Missed,1,NaN,NaN,0,h,MISS LaVine 26' 3PT Jump Shot,Missed Shot,Jump Shot,1,3,465,0022300033,NaN,0,7,2873,351,1554,653,CHI,ORL,home,after,0
292226,649,PT00M07.00S,4,1610612753,ORL,203914,Harris,G. Harris,0,0,0,,0,97.0,102.0,199,v,Harris Free Throw 1 of 2 (14 PTS),Free Throw,Free Throw 1 of 2,1,0,469,0022300033,-5.0,0,7,2873,351,1554,653,CHI,ORL,away,after,0
292227,650,PT00M07.00S,4,1610612753,ORL,203914,Harris,G. Harris,0,0,0,,0,97.0,103.0,200,v,Harris Free Throw 2 of 2 (15 PTS),Free Throw,Free Throw 2 of 2,1,0,470,0022300033,-6.0,0,7,2873,351,1554,653,CHI,ORL,away,after,0


### Assist Rate

### Shot Types

In [70]:
all_plays["actionType"].unique()

array(['period', 'Jump Ball', 'Made Shot', 'Turnover', '', 'Foul',
       'Free Throw', 'Rebound', 'Missed Shot', 'Timeout', 'Substitution',
       'Instant Replay', 'Violation', 'Ejection'], dtype=object)

In [71]:
assists = all_plays_fil.loc[
    (all_plays_fil["teamTricode"] != "")
    & (all_plays_fil["actionType"] == "Made Shot"),
    ["teamTricode", "description", "actionNumber", "game_id"]
].assign(
    assist=lambda row: row["description"].str.contains("AST")
)

In [72]:
shotchart_detail = pd.concat([
    nba_endpoints.shotchartdetail.ShotChartDetail(
        team_id="0",
        player_id="0",
        context_measure_simple = 'FGA',     
        season_type_all_star='Regular Season',
        season_nullable="2023-24"
    ).get_data_frames()[0],
    nba_endpoints.shotchartdetail.ShotChartDetail(
        team_id="0",
        player_id="0",
        context_measure_simple = 'FGA',     
        season_type_all_star='Regular Season',
        season_nullable="2024-25"
    ).get_data_frames()[0]
], axis=0)

KeyboardInterrupt: 

In [624]:
shotchart_detail.to_parquet("shotchart_detail.parquet")

In [153]:
shotchart_raw = pd.read_parquet("shotchart_detail.parquet")

In [154]:
shotchart_detail = all_plays.loc[
    (all_plays["isFieldGoal"]==1) | (all_plays["actionType"].isin(["Free Throw", "Turnover"]))
].merge(
    game_ids_20_point_leads,
    on="game_id"
).merge(
    games[["GAME_ID", "home_team", "away_team"]].drop_duplicates(),
    left_on="game_id",
    right_on="GAME_ID"
).assign(
    leading_team=lambda row: np.where(row["point_diff"]<0, "away", "home"),
    lead_20_half=lambda row: np.where(row["lead_20_seconds_elapsed"] <= 1440, "1H", "2H"),
    lead_20_quarter=lambda row: np.where(
        row["lead_20_seconds_elapsed"] <= 720, "1Q", np.where(
            row["lead_20_seconds_elapsed"] <= 1440, "2Q", np.where(
                row["lead_20_seconds_elapsed"] <= 2160, "3Q", "4Q"
            )
        ))
).drop("GAME_ID", axis=1).merge(
    shotchart_raw,
    left_on=["game_id", "personId", "actionNumber"],
    right_on=["GAME_ID", "PLAYER_ID", "GAME_EVENT_ID"]
).merge(
    assists,
    on=["game_id", "actionNumber", "teamTricode"],
    how="left"
)

In [75]:
shotchart_detail

,actionNumber,clock,period,teamId,teamTricode,personId,playerName,playerNameI,xLegacy,yLegacy,shotDistance,shotResult,isFieldGoal,scoreHome,scoreAway,pointsTotal,location,description_x,actionType,subType,videoAvailable,shotValue,actionId,game_id,point_diff,minutes,seconds,seconds_elapsed,lead_20_actionNumber,lead_20_seconds_elapsed,end_actionNumber,home_team,away_team,leading_team,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,description_y,assist
0,7,PT11M43.00S,1,1610612738,BOS,201950,Holiday,J. Holiday,43,7,4,Made,1,2.0,0.0,2,h,Holiday 4' Finger Roll Layup (2 PTS),Made Shot,Finger Roll Layup Shot,1,2,3,0022300118,2.0,11,43,17,210,956,646,BOS,IND,home,Shot Chart Detail,0022300118,7,201950,Jrue Holiday,1610612738,Boston Celtics,1,11,43,Made Shot,Finger Roll Layup Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,4,43,7,1,1,20231101,BOS,IND,Holiday 4' Finger Roll Layup (2 PTS),False
1,9,PT11M33.00S,1,1610612754,IND,1629614,Nembhard,A. Nembhard,-64,151,16,Made,1,2.0,2.0,4,v,Nembhard 16' Pullup Jump Shot (2 PTS),Made Shot,Pullup Jump shot,1,2,4,0022300118,0.0,11,33,27,210,956,646,BOS,IND,home,Shot Chart Detail,0022300118,9,1629614,Andrew Nembhard,1610612754,Indiana Pacers,1,11,33,Made Shot,Pullup Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-64,151,1,1,20231101,BOS,IND,Nembhard 16' Pullup Jump Shot (2 PTS),False
2,10,PT11M23.00S,1,1610612738,BOS,1628401,White,D. White,15,110,11,Made,1,4.0,2.0,6,h,White 11' Driving Floating Jump Shot (2 PTS),Made Shot,Driving Floating Jump Shot,1,2,5,0022300118,2.0,11,23,37,210,956,646,BOS,IND,home,Shot Chart Detail,0022300118,10,1628401,Derrick White,1610612738,Boston Celtics,1,11,23,Made Shot,Driving Floating Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,11,15,110,1,1,20231101,BOS,IND,White 11' Driving Floating Jump Shot (2 PTS),False
3,11,PT11M00.00S,1,1610612754,IND,1626167,Turner,M. Turner,34,48,6,Made,1,4.0,4.0,8,v,Turner 6' Layup (2 PTS) (Brown 1 AST),Made Shot,Layup Shot,1,2,6,0022300118,0.0,11,0,60,210,956,646,BOS,IND,home,Shot Chart Detail,0022300118,11,1626167,Myles Turner,1610612754,Indiana Pacers,1,11,0,Made Shot,Layup Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,5,34,48,1,1,20231101,BOS,IND,Turner 6' Layup (2 PTS) (Brown 1 AST),True
4,15,PT10M46.00S,1,1610612754,IND,1631097,Mathurin,B. Mathurin,7,29,3,Made,1,4.0,6.0,10,v,Mathurin 3' Running Finger Roll Layup (2 PTS) ...,Made Shot,Running Finger Roll Layup Shot,1,2,9,0022300118,-2.0,10,46,74,210,956,646,BOS,IND,away,Shot Chart Detail,0022300118,15,1631097,Bennedict Mathurin,1610612754,Indiana Pacers,1,10,46,Made Shot,Running Finger Roll Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,7,29,1,1,20231101,BOS,IND,Mathurin 3' Running Finger Roll Layup (2 PTS) ...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209907,621,PT00M26.10S,4,1610612741,CHI,1627936,Caruso,A. Caruso,-25,273,27,Made,1,97.0,96.0,193,h,Caruso 27' 3PT Jump Shot (18 PTS) (White 4 AST),Made Shot,Jump Shot,1,3,449,0022300033,1.0,0,26,2854,351,1554,653,CHI,ORL,home,Shot Chart Detail,0022300033,621,1627936,Alex Caruso,1610612741,Chicago Bulls,4,0,26,Made Shot,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,27,-25,273,1,1,20231117,CHI,ORL,Caruso 27' 3PT Jump Shot (18 PTS) (White 4 AST),True
209908,628,PT00M20.60S,4,1610612753,ORL,1630532,Wagner,F. Wagner,10,17,2,Made,1,97.0,98.0,195,v,F. Wagner 2' Driving Layup (20 PTS),Made Shot,Driving Layup Shot,1,2,453,0022300033,-1.0,0,20,2860,351,1554,653,CHI,ORL,away,Shot Chart Detail,0022300033,

In [155]:
shotchart_detail["lead_20_category"] = np.where(
    shotchart_detail["actionNumber"] < shotchart_detail["lead_20_actionNumber"], 'before', 'after'
)

In [77]:
assist_agg = shotchart_detail.groupby(
    ["teamTricode", "assist", "lead_20_category"]
).agg({
    "SHOT_MADE_FLAG": "sum"
}).reset_index().pivot_table(
    index="teamTricode",
    columns=["lead_20_category", "assist"],
    values=["SHOT_MADE_FLAG"],
    aggfunc="sum"
).reset_index()

In [156]:
shotchart_agg = shotchart_detail.groupby(
    ["teamTricode", "SHOT_ZONE_BASIC", "lead_20_category", "lead_20_half"]
).agg({
    "SHOT_ATTEMPTED_FLAG": "sum",
    "SHOT_MADE_FLAG": "sum"
}).reset_index().pivot_table(
    index="teamTricode",
    columns=["lead_20_half", "lead_20_category", "SHOT_ZONE_BASIC"],
    values=["SHOT_MADE_FLAG", "SHOT_ATTEMPTED_FLAG"],
    aggfunc="sum"
).reset_index()

In [79]:
assist_agg.columns=["team", "no_assist_after", "assist_after", "no_assist_before", "assist_before"]

In [80]:
assist_agg["assist_rate_before"] = assist_agg["assist_before"]/(assist_agg["assist_before"]+assist_agg["no_assist_before"])
assist_agg["assist_rate_after"] = assist_agg["assist_after"]/(assist_agg["assist_after"]+assist_agg["no_assist_after"])

In [81]:
fig=dumbbell_plot(
    assist_agg, 
    "team", "assist_rate_before", "assist_rate_after", 
    "Assist Rate - Before", "Assist Rate - After", "Assist Rate", 
    "Assist Rate - Before v. After 20-point lead", #axis_range=[0.34, 0.52]
)
fig.add_shape(
    type="rect",
    xref="paper", x0=-0.05, x1=1,
    yref="paper", y0=0.3825, y1=0.41,
    line=dict(color="green"),
    opacity=0.5
)

In [82]:
assist_agg

,team,no_assist_after,assist_after,no_assist_before,assist_before,assist_rate_before,assist_rate_after
0,ATL,501,910,592,1201,0.669827,0.644933
1,BKN,415,801,633,1177,0.650276,0.658717
2,BOS,638,1078,871,1476,0.628888,0.628205
3,CHA,523,860,578,973,0.627337,0.621837
4,CHI,492,793,545,1065,0.661491,0.617121
5,CLE,526,1019,621,1150,0.649351,0.659547
6,DAL,588,944,791,1090,0.579479,0.616188
7,DEN,471,984,648,1452,0.691429,0.676289
8,DET,530,904,606,1067,0.637776,0.630404
9,GSW,366,803,419,1064,0.717465,0.686912


In [159]:
shotchart_agg.transpose().to_clipboard()

In [83]:
shotchart_agg.columns = [
    "team", 
    "above_the_break_3_after_att",
    "backcourt_after_att",
    "in_the_paint_non_ra_after_att",
    "left_corner_3_after_att",
    "mid_range_after_att",
    "restricted_area_after_att",
    "right_corner_3_after_att",
    "above_the_break_3_before_att",
    "backcourt_before_att",
    "in_the_paint_non_ra_before_att",
    "left_corner_3_before_att",
    "mid_range_before_att",
    "restricted_area_before_att",
    "right_corner_3_before_att",
    "above_the_break_3_after_make",
    "backcourt_after_make",
    "in_the_paint_non_ra_after_make",
    "left_corner_3_after_make",
    "mid_range_after_make",
    "restricted_area_after_make",
    "right_corner_3_after_make",
    "above_the_break_3_before_make",
    "backcourt_before_make",
    "in_the_paint_non_ra_before_make",
    "left_corner_3_before_make",
    "mid_range_before_make",
    "restricted_area_before_make",
    "right_corner_3_before_make"
]

In [84]:
shotchart_agg["total_before_att"] = shotchart_agg["above_the_break_3_before_att"] + \
shotchart_agg["backcourt_before_att"] + \
shotchart_agg["in_the_paint_non_ra_before_att"] + \
shotchart_agg["left_corner_3_before_att"] + \
shotchart_agg["mid_range_before_att"] + \
shotchart_agg["restricted_area_before_att"] + \
shotchart_agg["right_corner_3_before_att"]

shotchart_agg["total_after_att"] = shotchart_agg["above_the_break_3_after_att"] + \
shotchart_agg["backcourt_after_att"] + \
shotchart_agg["in_the_paint_non_ra_after_att"] + \
shotchart_agg["left_corner_3_after_att"] + \
shotchart_agg["mid_range_after_att"] + \
shotchart_agg["restricted_area_after_att"] + \
shotchart_agg["right_corner_3_after_att"]

shotchart_agg["total_before_make"] = shotchart_agg["above_the_break_3_before_make"] + \
shotchart_agg["backcourt_before_make"] + \
shotchart_agg["in_the_paint_non_ra_before_make"] + \
shotchart_agg["left_corner_3_before_make"] + \
shotchart_agg["mid_range_before_make"] + \
shotchart_agg["restricted_area_before_make"] + \
shotchart_agg["right_corner_3_before_make"]

shotchart_agg["total_after_make"] = shotchart_agg["above_the_break_3_after_make"] + \
shotchart_agg["backcourt_after_make"] + \
shotchart_agg["in_the_paint_non_ra_after_make"] + \
shotchart_agg["left_corner_3_after_make"] + \
shotchart_agg["mid_range_after_make"] + \
shotchart_agg["restricted_area_after_make"] + \
shotchart_agg["right_corner_3_after_make"]

shotchart_agg["above_the_break_3_before_rate"] = shotchart_agg["above_the_break_3_before_att"]/shotchart_agg["total_before_att"]
shotchart_agg["backcourt_before_rate"] = shotchart_agg["backcourt_before_att"]/shotchart_agg["total_before_att"]
shotchart_agg["in_the_paint_non_ra_before_rate"] = shotchart_agg["in_the_paint_non_ra_before_att"]/shotchart_agg["total_before_att"]
shotchart_agg["left_corner_3_before_rate"] = shotchart_agg["left_corner_3_before_att"]/shotchart_agg["total_before_att"]
shotchart_agg["mid_range_before_rate"] = shotchart_agg["mid_range_before_att"]/shotchart_agg["total_before_att"]
shotchart_agg["restricted_area_before_rate"] = shotchart_agg["restricted_area_before_att"]/shotchart_agg["total_before_att"]
shotchart_agg["right_corner_3_before_rate"] = shotchart_agg["right_corner_3_before_att"]/shotchart_agg["total_before_att"]

shotchart_agg["above_the_break_3_after_rate"] = shotchart_agg["above_the_break_3_after_att"]/shotchart_agg["total_after_att"]
shotchart_agg["backcourt_after_rate"] = shotchart_agg["backcourt_after_att"]/shotchart_agg["total_after_att"]
shotchart_agg["in_the_paint_non_ra_after_rate"] = shotchart_agg["in_the_paint_non_ra_after_att"]/shotchart_agg["total_after_att"]
shotchart_agg["left_corner_3_after_rate"] = shotchart_agg["left_corner_3_after_att"]/shotchart_agg["total_after_att"]
shotchart_agg["mid_range_after_rate"] = shotchart_agg["mid_range_after_att"]/shotchart_agg["total_after_att"]
shotchart_agg["restricted_area_after_rate"] = shotchart_agg["restricted_area_after_att"]/shotchart_agg["total_after_att"]
shotchart_agg["right_corner_3_after_rate"] = shotchart_agg["right_corner_3_after_att"]/shotchart_agg["total_after_att"]

shotchart_agg["above_the_break_3_before_pct"]= shotchart_agg["above_the_break_3_before_make"]/shotchart_agg["above_the_break_3_before_att"]
shotchart_agg["backcourt_before_pct"]= shotchart_agg["backcourt_before_make"]/shotchart_agg["backcourt_before_att"]
shotchart_agg["in_the_paint_non_ra_before_pct"]= shotchart_agg["in_the_paint_non_ra_before_make"]/shotchart_agg["in_the_paint_non_ra_before_att"]
shotchart_agg["left_corner_3_before_pct"]= shotchart_agg["left_corner_3_before_make"]/shotchart_agg["left_corner_3_before_att"]
shotchart_agg["mid_range_before_pct"]= shotchart_agg["mid_range_before_make"]/shotchart_agg["mid_range_before_att"]
shotchart_agg["restricted_area_before_pct"]= shotchart_agg["restricted_area_before_make"]/shotchart_agg["restricted_area_before_att"]
shotchart_agg["right_corner_3_before_pct"]= shotchart_agg["right_corner_3_before_make"]/shotchart_agg["right_corner_3_before_att"]

shotchart_agg["above_the_break_3_after_pct"]= shotchart_agg["above_the_break_3_after_make"]/shotchart_agg["above_the_break_3_after_att"]
shotchart_agg["backcourt_after_pct"]= shotchart_agg["backcourt_after_make"]/shotchart_agg["backcourt_after_att"]
shotchart_agg["in_the_paint_non_ra_after_pct"]= shotchart_agg["in_the_paint_non_ra_after_make"]/shotchart_agg["in_the_paint_non_ra_after_att"]
shotchart_agg["left_corner_3_after_pct"]= shotchart_agg["left_corner_3_after_make"]/shotchart_agg["left_corner_3_after_att"]
shotchart_agg["mid_range_after_pct"]= shotchart_agg["mid_range_after_make"]/shotchart_agg["mid_range_after_att"]
shotchart_agg["restricted_area_after_pct"]= shotchart_agg["restricted_area_after_make"]/shotchart_agg["restricted_area_after_att"]
shotchart_agg["right_corner_3_after_pct"]= shotchart_agg["right_corner_3_after_make"]/shotchart_agg["right_corner_3_after_att"]

In [85]:
shotchart_agg.loc[shotchart_agg["team"]=="CLE"].transpose().reset_index().to_clipboard(index=False)

# Viz

In [86]:
def degree2rad(degrees: int) -> float:
    return degrees*np.pi/180

# segment types: open, segment, sector
def disk_part(
    center: list=[0,0], radius: int=1, start_angle: int=0, end_angle: int=90, n: int=50, seg_type: str='open'
) -> str:
    sta = degree2rad(start_angle)
    ea = degree2rad(end_angle)
    t = np.linspace(sta, ea, n)
    x = center[0] + radius*np.cos(t)
    y = center[1] +radius*np.sin(t)
    path = f"M {x[0]},{y[0]}"
    for xc, yc in zip(x[1:], y[1:]):
        path += f" L{xc},{yc}"
    if seg_type == 'open':
        return path
    elif seg_type == 'segment':
        return path + " Z"
    else: #disk sector
        return path + f" L{center[0]},{center[1]} Z" #sector 
    
    
def draw_court(
    fig: go.Figure = None, line_color: str="black", line_width: int=2
) -> go.Figure:

    if fig == None:
        fig = go.Figure()


    # Layout
    fig.update_layout(
        xaxis=dict(range=(-300, 300)),
        yaxis=dict(range=(-100, 500)),
        height=1000,
        width=1000*(11/12),
        plot_bgcolor="white"
    )
    
    # Outer Lines
    fig.add_shape(
        type="rect",
        xref="x", yref="y",
        x0=-250, y0=-47.5,
        x1=250, y1=422.5,
        line=dict(
            color=line_color,
            width=line_width,
        ),
        fillcolor=None,
    )
    
    # Center Court
    # # Outer Arc
    fig.add_shape(
        type="path",
        path=disk_part(
            center=[0,422.5],
            radius=60,
            start_angle=0,
            end_angle=-180,
            n=50,
            seg_type='open'
        ),
        fillcolor=None,
        line=dict(color=line_color, dash="solid")
    )
    # # Inner Arc
    fig.add_shape(
        type="path",
        path=disk_part(
            center=[0,422.5],
            radius=20,
            start_angle=0,
            end_angle=-180,
            n=50,
            seg_type='open'
        ),
        fillcolor=None,
        line=dict(color=line_color, dash="solid")
    )
    
    # Three point line
    # # Corner 3pt Lines
    fig.add_shape(
        type="line",
        xref="x", yref="y",
        x0=-220, y0=-47.5,
        x1=-220, y1=92.5,
        line=dict(
            color=line_color,
            width=line_width,
        ),
        fillcolor=None,
    )
    fig.add_shape(
        type="line",
        xref="x", yref="y",
        x0=220, y0=-47.5,
        x1=220, y1=92.5,
        line=dict(
            color=line_color,
            width=line_width,
        ),
        fillcolor=None,
    )

    # # 3pt Arc
    fig.add_shape(
        type="path",
        path=disk_part(
            center=[0,0],
            radius=475/2,
            start_angle=22.5,
            end_angle=157.5,
            n=50,
            seg_type='open'
        ),
        fillcolor=None,
        line=dict(color=line_color, dash="solid")
    )
    
    # The Paint
    # # Outer Box
    fig.add_shape(
        type="rect",
        xref="x", yref="y",
        x0=-80, y0=-47.5,
        x1=80, y1=142.5,
        line=dict(
            color=line_color,
            width=line_width,
        ),
        fillcolor=None,
    )

    # # Inner Box
    fig.add_shape(
        type="rect",
        xref="x", yref="y",
        x0=-60, y0=-47.5,
        x1=60, y1=142.5,
        line=dict(
            color=line_color,
            width=line_width,
        ),
        fillcolor=None,
    )
    
    # Free Throw Circle
    # # Top Arc
    fig.add_shape(
        type="path",
        path=disk_part(
            center=[0,142.5],
            radius=60,
            start_angle=0,
            end_angle=180,
            n=50,
            seg_type='open'
        ),
        fillcolor=None,
        line=dict(color=line_color, dash="solid")
    )
    #     top_free_throw = Arc((0, 142.5), 120, 120, theta1=0, theta2=180,
    #                          linewidth=line_width, color=color, fill=False)


    # # Bottom arc
    fig.add_shape(
        type="path",
        path=disk_part(
            center=[0,142.5],
            radius=60,
            start_angle=0,
            end_angle=-180,
            n=50,
            seg_type='open'
        ),
        fillcolor=None,
        line=dict(color=line_color, dash="dash")
    )

    # Restricted Zone Arc
    fig.add_shape(
        type="path",
        path=disk_part(
            center=[0,0],
            radius=40,
            start_angle=0,
            end_angle=180,
            n=50,
            seg_type='open'
        ),
        fillcolor=None,
        line=dict(color=line_color, dash="solid")
    )

    # Hoop
    fig.add_shape(
        type="circle",
        xref="x", yref="y",
        x0=-7.5, y0=-7.5, x1=7.5, y1=7.5,
        line=dict(color=line_color, width=line_width),
        fillcolor=None
    )

    # Backboard
    fig.add_shape(
        type="line",
        xref="x", yref="y",
        x0=-30, y0=-7.5,
        x1=30, y1=-7.5,
        line=dict(
            color=line_color,
            width=line_width,
        ),
        fillcolor=None,
    )

    return fig

In [87]:
def get_hexbin_attributes(hexbin):
    paths = hexbin.get_paths()
    points_codes = list(paths[0].iter_segments())#path[0].iter_segments() is a generator 
    prototypical_hexagon = [item[0] for item in points_codes]
    return prototypical_hexagon, hexbin.get_offsets(), hexbin.get_facecolors(), hexbin.get_array()

def pl_cell_color(mpl_facecolors):
     
    return [ f'rgb({int(R*255)}, {int(G*255)}, {int(B*255)})' for (R, G, B, A) in mpl_facecolors]

def make_hexagon(prototypical_hex, offset, fillcolor, linecolor=None):
   
    new_hex_vertices = [vertex + offset for vertex in prototypical_hex]
    vertices = np.asarray(new_hex_vertices[:-1])
    # hexagon center
    center=np.mean(vertices, axis=0)
    if linecolor is None:
        linecolor = fillcolor
    #define the SVG-type path:    
    path = 'M '
    for vert in new_hex_vertices:
        path +=  f'{vert[0]}, {vert[1]} L' 
    return  dict(type='path',
                 line=dict(color=linecolor, 
                           width=0.5),
                 path=  path[:-2],
                 fillcolor=fillcolor, opacity=0.8
                ), center 

def mpl_to_plotly(cmap, N):
    h = 1.0/(N-1)
    pl_colorscale = []
    for k in range(N):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([round(k*h,2), f'rgb({C[0]}, {C[1]}, {C[2]})'])
    return pl_colorscale

In [88]:
tatum_shots = shot_profile.loc[(shot_profile["playerName"]=='Tatum') & (shot_profile["isFieldGoal"]==1)]

In [89]:
plt.figure(figsize=(0.05,0.05))
plt.axis('off')
HB = plt.hexbin(
    x=tatum_shots["xLegacy"], 
    y=tatum_shots["yLegacy"], 
#     C=tatum_shots["yLegacy"], 
    gridsize=12, 
    cmap=cmocean.cm.algae , 
    mincnt=5
) # cmocean.cm.algae is a cmocean colormap

In [90]:
hexagon_vertices, offsets, mpl_facecolors, counts = get_hexbin_attributes(HB)

In [91]:
counts

masked_array(data=[  6.,  21.,  35.,   7.,   5.,  36.,   7.,  15.,  49.,
                    22.,   9.,   7.,  42.,   9.,  45.,   5.,  25.,  22.,
                    29.,  11.,  17.,  26.,   7.,   5.,  43.,  16.,  16.,
                    10.,  12.,  10.,   6.,  13.,   7.,  16.,   9.,  22.,
                    11.,   8.,   7.,  65.,  11.,   5.,  33.,   5.,   6.,
                     7.,   7., 106.,  10.,  13., 230.,  16.,  10.,  15.,
                    33.,  10.,   6.,  10.,  16.,  11.,   8.,  12.,  52.,
                     5.,  20.,   5.,   6.],
             mask=False,
       fill_value=1e+20)

In [92]:
cell_color = pl_cell_color(mpl_facecolors)

In [93]:
pl_algae = mpl_to_plotly(cmocean.cm.algae, 11)
pl_algae

[[0.0, 'rgb(214, 249, 207)'],
 [0.1, 'rgb(183, 226, 171)'],
 [0.2, 'rgb(149, 205, 137)'],
 [0.3, 'rgb(114, 186, 108)'],
 [0.4, 'rgb(67, 168, 85)'],
 [0.5, 'rgb(17, 147, 79)'],
 [0.6, 'rgb(9, 125, 74)'],
 [0.7, 'rgb(21, 102, 64)'],
 [0.8, 'rgb(25, 80, 52)'],
 [0.9, 'rgb(24, 57, 36)'],
 [1.0, 'rgb(17, 36, 20)']]

In [94]:
shapes = []
centers = []
for k in range(len(offsets)):
    shape, center = make_hexagon(hexagon_vertices, offsets[k], cell_color[k])
    shapes.append(shape)
    centers.append(center)

In [95]:
X, Y = zip(*centers)

#define  text to be  displayed on hovering the mouse over the cells
text = [f'x: {round(X[k],2)}<br>y: {round(Y[k],2)}<br>counts: {int(counts[k])}' for k in range(len(X))]

In [96]:
trace = go.Scatter(
    x=list(X),
    y=list(Y),
    mode='markers',
    marker=dict(size=0.5,
                color=counts,
                colorscale=pl_algae,
                showscale=True,
                colorbar=dict(
                    thickness=20,
                    ticklen=4
                )),
    text=text,
    hoverinfo='text'
)

In [97]:
axis = dict(showgrid=False,
           showline=False,
           zeroline=False,
           ticklen=4 
           )

layout = go.Layout(title='Hexbin plot',
                   width=530, height=550,
                   xaxis=axis,
                   yaxis=axis,
                   hovermode='closest',
                   shapes=shapes,
                   plot_bgcolor='black')

In [98]:
fig = draw_court()
fig.add_trace(
    go.Scatter(
        x=tatum_shots["xLegacy"],
        y=tatum_shots["yLegacy"],
        mode="markers",
        marker=dict(color=np.where(tatum_shots["shotResult_int"],'blue','red'))
    )
)
# fig.add_trace(
#     trace
# )

# fig.update_layout(shapes=shapes, showlegend=False)

In [99]:
viz_df = dff.groupby("start_winning_team").agg({
    "game_id": "nunique",
    "start_winning_team_won": "nunique",
    "diff_of_diffs": "mean",
    "clutch_definition_seconds_elapsed": "mean",
    "clutch_definition_game_id": "nunique",
    "seconds_to_clutch_definition": "mean",
    "within_2_possessions_seconds_elapsed": "mean",
    "within_2_possessions_game_id": "nunique",
    "seconds_to_within_2_possessions": "mean",
    "lead_20_seconds_elapsed": "mean",
    "tied_lost_lead_game_id": "nunique",
    "seconds_to_tied_lost_lead": "mean"
}).reset_index().assign(
    win_pct=lambda row: row["start_winning_team_won"]/row["game_id"],
    losses=lambda row: row["game_id"] - row["start_winning_team_won"],
    mins_elapsed=lambda row: row["lead_20_seconds_elapsed"]/60,
    mins_to_clutch_definition=lambda row: row["seconds_to_within_2_possessions"]/60,
    clutch_definition_pct=lambda row: row["clutch_definition_game_id"]/row["game_id"],
    mins_to_within_2_possessions=lambda row: row["seconds_to_within_2_possessions"]/60,
    within_2_possessions_pct=lambda row: row["within_2_possessions_game_id"]/row["game_id"],
    mins_to_tied_lost_lead=lambda row: row["seconds_to_tied_lost_lead"]/60,
    tied_lost_lead_pct=lambda row: row["tied_lost_lead_game_id"]/row["game_id"]
).sort_values(
    "clutch_definition_pct", 
    ascending=True
)

In [100]:
dff.loc[dff["game_id"]=='0022300479']

,game_id,scoreHome,scoreAway,point_diff,seconds_elapsed,home_team,away_team,actionNumber,lead_20_actionNumber,lead_20_seconds_elapsed,tied_lost_lead_game_id,within_2_possessions_game_id,clutch_definition_game_id,tied_lost_lead_actionNumber,tied_lost_lead_seconds_elapsed,seconds_to_tied_lost_lead,within_2_possessions_actionNumber,within_2_possessions_seconds_elapsed,seconds_to_within_2_possessions,clutch_definition_actionNumber,clutch_definition_seconds_elapsed,seconds_to_clutch_definition,end_actionNumber,start_point_diff,end_point_diff,start_winning_team,end_winning_team,diff_of_diffs,start_winning_team_won
136321,0022300479,0.0,0.0,0.0,0,BOS,UTA,2,182,795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,713,20.0,29.0,BOS,BOS,9.0,0022300479
136322,0022300479,1.0,0.0,1.0,31,BOS,UTA,22,182,795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,713,20.0,29.0,BOS,BOS,9.0,0022300479
136323,0022300479,4.0,0.0,4.0,56,BOS,UTA,26,182,795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,713,20.0,29.0,BOS,BOS,9.0,0022300479
136324,0022300479,6.0,0.0,6.0,86,BOS,UTA,30,182,795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,713,20.0,29.0,BOS,BOS,9.0,0022300479
136325,0022300479,8.0,0.0,8.0,107,BOS,UTA,34,182,795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,713,20.0,29.0,BOS,BOS,9.0,0022300479
136326,0022300479,10.0,0.0,10.0,138,BOS,UTA,41,182,795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,713,20.0,29.0,BOS,BOS,9.0,0022300479
136327,0022300479,10.0,1.0,9.0,163,BOS,UTA,48,182,795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,713,20.0,29.0,BOS,BOS,9.0,0022300479
136328,0022300479,10.0,2.0,8.0,163,BOS,UTA,49,182,795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,713,20.0,29.0,BOS,BOS,9.0,0022300479
136329,0022300479,12.0,2.0,10.0,217,BOS,UTA,58,182,795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,713,20.0,29.0,BOS,BOS,9.0,0022300479
136330,0022300479,12.0,4.0,8.0,241,BOS,UTA,62,182,795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,713,20.0,29.0,BOS,BOS,9.0,0022300479


In [101]:
dff.loc[
    pd.isnull(dff["within_2_possessions_game_id"]) 
    & pd.isnull(dff["clutch_definition_game_id"])
    & (dff["start_winning_team"]=="BOS")
    & (dff["end_point_diff"].abs() > 1),
    ["game_id", "home_team", "away_team", "end_point_diff"]
].drop_duplicates().sort_values("end_point_diff")

,game_id,home_team,away_team,end_point_diff
104535,0022400604,GSW,BOS,-40.0
143976,0022300389,LAC,BOS,-37.0
107552,0022300630,MIA,BOS,-33.0
109677,0022300448,SAS,BOS,-33.0
36923,0022400157,ATL,BOS,-30.0
1700,0022401058,PHX,BOS,-30.0
116550,0022400748,NYK,BOS,-27.0
88949,0022301011,DET,BOS,-27.0
109163,0022300980,WAS,BOS,-26.0
121529,0022300373,SAC,BOS,-25.0


In [102]:
games.loc[games["GAME_ID"]=='0022301227']

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,AVAILABLE_FLAG,home,team_1,team_2,home_team,away_team
1818,2023-24,1610612738,BOS,Boston Celtics,0022301227,2023-12-08T00:00:00,BOS vs. NYK,W,48.0,48,91,0.527,19,47,0.404,18,21,0.857,8,26,34,24,8.0,8,5,3,19,20,133,10.0,1,1,1,1,119,315,840,417,128,84,772,926,1194,545,1669,2189,2268,1586,117,839,1003,382,1208,790,160,598,1,True,BOS,NYK,BOS,NYK
1839,2023-24,1610612752,NYK,New York Knicks,0022301227,2023-12-08T00:00:00,NYK @ BOS,L,48.0,44,88,0.500,15,37,0.405,20,27,0.741,12,32,44,28,14.0,4,3,5,20,19,123,-10.0,1,1231,1231,1231,119,791,1262,757,574,851,751,640,497,1661,703,1277,1055,852,1228,2100,1752,1096,1442,1020,583,1812,1,False,NYK,BOS,BOS,NYK


## Win %

In [103]:
viz_df

,start_winning_team,game_id,start_winning_team_won,diff_of_diffs,clutch_definition_seconds_elapsed,clutch_definition_game_id,seconds_to_clutch_definition,within_2_possessions_seconds_elapsed,within_2_possessions_game_id,seconds_to_within_2_possessions,lead_20_seconds_elapsed,tied_lost_lead_game_id,seconds_to_tied_lost_lead,win_pct,losses,mins_elapsed,mins_to_clutch_definition,clutch_definition_pct,mins_to_within_2_possessions,within_2_possessions_pct,mins_to_tied_lost_lead,tied_lost_lead_pct
5,CLE,54,52,-0.866178,2152.000000,1,1491.000000,2173.743079,7,1176.524917,1487.152311,3,1571.329897,0.962963,2,24.785872,19.608749,0.018519,19.608749,0.129630,26.188832,0.055556
13,LAL,45,45,-4.097565,1825.000000,1,940.000000,2162.011843,10,997.450777,1579.777984,1,1366.000000,1.000000,0,26.329633,16.624180,0.022222,16.624180,0.222222,22.766667,0.022222
22,PHI,27,25,-1.605586,2047.000000,1,1031.000000,2337.624060,5,770.485714,1673.952693,1,1107.000000,0.925926,2,27.899212,12.841429,0.037037,12.841429,0.185185,18.450000,0.037037
7,DEN,50,49,-1.817331,1931.000000,2,1215.197802,2283.952498,9,812.361998,1692.115036,2,1101.052632,0.980000,1,28.201917,13.539367,0.040000,13.539367,0.180000,18.350877,0.040000
11,IND,39,39,1.576305,1887.945652,2,647.891304,2119.811676,8,665.258945,1884.665830,2,956.712230,1.000000,0,31.411097,11.087649,0.051282,11.087649,0.205128,15.945204,0.051282
23,PHX,39,37,-4.810065,1818.800000,2,1134.753846,2320.217850,8,1080.599808,1568.965097,2,1106.351351,0.948718,2,26.149418,18.009997,0.051282,18.009997,0.205128,18.439189,0.051282
19,NYK,53,52,-0.278592,1958.277174,3,910.584239,2213.370306,9,858.431441,1672.502644,1,1623.000000,0.981132,1,27.875044,14.307191,0.056604,14.307191,0.169811,27.050000,0.018868
17,MIN,53,50,-1.767765,1722.565327,3,632.045226,2124.736178,8,696.074685,1880.317170,5,1040.758621,0.943396,3,31.338619,11.601245,0.056604,11.601245,0.150943,17.345977,0.094340
25,SAC,43,39,-3.174471,1818.557252,3,653.819338,2012.404239,10,726.141559,1718.363045,5,1055.410575,0.906977,4,28.639384,12.102359,0.069767,12.102359,0.232558,17.590176,0.116279
12,LAC,54,49,-2.617885,1610.928983,4,599.604607,1930.936455,7,539.685619,1641.016758,6,836.292994,0.907407,5,27.350279,8.994760,0.074074,8.994760,0.129630,13.938217,0.111111


In [104]:
import plotly.io as pio
pio.renderers.default = 'iframe_connected'

In [105]:
viz_df.sort_values("game_id", ascending=False).head()

,start_winning_team,game_id,start_winning_team_won,diff_of_diffs,clutch_definition_seconds_elapsed,clutch_definition_game_id,seconds_to_clutch_definition,within_2_possessions_seconds_elapsed,within_2_possessions_game_id,seconds_to_within_2_possessions,lead_20_seconds_elapsed,tied_lost_lead_game_id,seconds_to_tied_lost_lead,win_pct,losses,mins_elapsed,mins_to_clutch_definition,clutch_definition_pct,mins_to_within_2_possessions,within_2_possessions_pct,mins_to_tied_lost_lead,tied_lost_lead_pct
2,BOS,80,77,-0.316893,1807.755287,8,786.393756,2049.373219,20,721.208384,1611.429653,8,1127.818182,0.962500,3,26.857161,12.020140,0.100000,12.020140,0.250000,18.796970,0.100000
20,OKC,79,77,0.926545,1915.872748,7,566.757883,1983.286830,14,759.616071,1707.059021,5,1122.329866,0.974684,2,28.450984,12.660268,0.088608,12.660268,0.177215,18.705498,0.063291
12,LAC,54,49,-2.617885,1610.928983,4,599.604607,1930.936455,7,539.685619,1641.016758,6,836.292994,0.907407,5,27.350279,8.994760,0.074074,8.994760,0.129630,13.938217,0.111111
5,CLE,54,52,-0.866178,2152.000000,1,1491.000000,2173.743079,7,1176.524917,1487.152311,3,1571.329897,0.962963,2,24.785872,19.608749,0.018519,19.608749,0.129630,26.188832,0.055556
19,NYK,53,52,-0.278592,1958.277174,3,910.584239,2213.370306,9,858.431441,1672.502644,1,1623.000000,0.981132,1,27.875044,14.307191,0.056604,14.307191,0.169811,27.050000,0.018868


In [106]:
viz_df

,start_winning_team,game_id,start_winning_team_won,diff_of_diffs,clutch_definition_seconds_elapsed,clutch_definition_game_id,seconds_to_clutch_definition,within_2_possessions_seconds_elapsed,within_2_possessions_game_id,seconds_to_within_2_possessions,lead_20_seconds_elapsed,tied_lost_lead_game_id,seconds_to_tied_lost_lead,win_pct,losses,mins_elapsed,mins_to_clutch_definition,clutch_definition_pct,mins_to_within_2_possessions,within_2_possessions_pct,mins_to_tied_lost_lead,tied_lost_lead_pct
5,CLE,54,52,-0.866178,2152.000000,1,1491.000000,2173.743079,7,1176.524917,1487.152311,3,1571.329897,0.962963,2,24.785872,19.608749,0.018519,19.608749,0.129630,26.188832,0.055556
13,LAL,45,45,-4.097565,1825.000000,1,940.000000,2162.011843,10,997.450777,1579.777984,1,1366.000000,1.000000,0,26.329633,16.624180,0.022222,16.624180,0.222222,22.766667,0.022222
22,PHI,27,25,-1.605586,2047.000000,1,1031.000000,2337.624060,5,770.485714,1673.952693,1,1107.000000,0.925926,2,27.899212,12.841429,0.037037,12.841429,0.185185,18.450000,0.037037
7,DEN,50,49,-1.817331,1931.000000,2,1215.197802,2283.952498,9,812.361998,1692.115036,2,1101.052632,0.980000,1,28.201917,13.539367,0.040000,13.539367,0.180000,18.350877,0.040000
11,IND,39,39,1.576305,1887.945652,2,647.891304,2119.811676,8,665.258945,1884.665830,2,956.712230,1.000000,0,31.411097,11.087649,0.051282,11.087649,0.205128,15.945204,0.051282
23,PHX,39,37,-4.810065,1818.800000,2,1134.753846,2320.217850,8,1080.599808,1568.965097,2,1106.351351,0.948718,2,26.149418,18.009997,0.051282,18.009997,0.205128,18.439189,0.051282
19,NYK,53,52,-0.278592,1958.277174,3,910.584239,2213.370306,9,858.431441,1672.502644,1,1623.000000,0.981132,1,27.875044,14.307191,0.056604,14.307191,0.169811,27.050000,0.018868
17,MIN,53,50,-1.767765,1722.565327,3,632.045226,2124.736178,8,696.074685,1880.317170,5,1040.758621,0.943396,3,31.338619,11.601245,0.056604,11.601245,0.150943,17.345977,0.094340
25,SAC,43,39,-3.174471,1818.557252,3,653.819338,2012.404239,10,726.141559,1718.363045,5,1055.410575,0.906977,4,28.639384,12.102359,0.069767,12.102359,0.232558,17.590176,0.116279
12,LAC,54,49,-2.617885,1610.928983,4,599.604607,1930.936455,7,539.685619,1641.016758,6,836.292994,0.907407,5,27.350279,8.994760,0.074074,8.994760,0.129630,13.938217,0.111111


In [107]:
viz_df["start_winning_team_won"].sum()/viz_df["game_id"].sum()

0.9429301533219762

In [108]:
average = viz_df["game_id"].sum()/30

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=viz_df["start_winning_team"],
        y=viz_df["start_winning_team_won"],
        name="Wins",
        orientation="v",
        marker=dict(
            color="mediumseagreen", 
            opacity=[1 if el=="BOS" else 0.6 for el in viz_df["start_winning_team"].unique()]
        )
    )
)
fig.add_trace(
    go.Bar(
        x=viz_df["start_winning_team"],
        y=viz_df["losses"],
        name="Losses",
        text=viz_df["game_id"],
        textposition="outside",
        orientation="v",
        marker=dict(
            color="peru", 
            opacity=[1 if el=="BOS" else 0.6 for el in viz_df["start_winning_team"].unique()]
        )
    )
)
fig.add_hline(
    y=average,
    line=dict(dash="dot"),
    opacity=0.3
)
fig.add_annotation(
    xref="paper",
    x=0.9,
    y=average+2.5,
    text=f"<b>Average: {int(round(average,0))}</b>",
    font=dict(size=12),
    showarrow=False
)

fig.update_layout(
    barmode='stack', 
    xaxis={'categoryorder':'total descending'},
    yaxis=dict(visible=False),
    plot_bgcolor="white",
    title="Total games with 20-point lead - '23-24 & '24-25"
)

In [109]:
average = viz_df["start_winning_team_won"].sum()/viz_df["game_id"].sum()

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=viz_df["start_winning_team"],
        y=viz_df["start_winning_team_won"]/viz_df["game_id"],
        name="Wins",
        orientation="v",
        marker=dict(
            color="mediumseagreen", 
            opacity=[1 if el=="BOS" else 0.6 for el in viz_df["start_winning_team"].unique()]
        ),
        text=viz_df["start_winning_team_won"]/viz_df["game_id"],
        textposition="outside",
        texttemplate="%{y:.3f}"
    )
)
fig.add_hline(
    y=average,
    line=dict(dash="dot"),
    opacity=0.3
)
fig.add_annotation(
    xref="paper",
    x=0.98,
    y=average+.015,
    text=f"<b>Average: {round(average,3)}</b>",
    font=dict(size=12),
    showarrow=False
)

fig.update_layout(
    barmode='stack', 
    xaxis={'categoryorder':'total descending'},
    yaxis=dict(visible=False, range=[.7, 1.05]),
    plot_bgcolor="white",
    title="Win% of Games With 20-point lead - '23-24 & '24-25"
)

In [110]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=viz_df["start_winning_team"],
        y=(viz_df["game_id"] - viz_df["start_winning_team_won"]),
        name="Wins",
        orientation="v",
        marker=dict(
            color="peru", 
            opacity=[1 if el=="BOS" else 0.6 for el in viz_df["start_winning_team"].unique()]
        ),
        text=(viz_df["game_id"] - viz_df["start_winning_team_won"]),
        textposition="outside"
    )
)

fig.update_layout(
    barmode='stack', 
    xaxis={'categoryorder':'total ascending'},
    yaxis=dict(visible=False),
    plot_bgcolor="white",
    title="Win% of Games With 20-point lead - '23-24 & '24-25"
)

## Point Differential

In [111]:
## calculate quartiles as outlined in the plotly documentation 
def get_percentile(data, p):
    data.sort()
    n = len(data)
    x = n*p + 0.5
    x1, x2 = floor(x), ceil(x)
    y1, y2 = data[x1-1], data[x2-1] # account for zero-indexing
    if y2==y1:
        return y1
    else:
        return round(y1 + ((x - x1) / (x2 - x1))*(y2 - y1), 2)

In [112]:
dff.groupby(["start_winning_team", "game_id"]).agg(
    {
        "diff_of_diffs": "min",
        "start_winning_team_won": "min"
    }
)["diff_of_diffs"].mean()

-2.4207836456558773

In [113]:
average = dff.groupby(["start_winning_team", "game_id"]).agg(
    {
        "diff_of_diffs": "min",
        "start_winning_team_won": "min"
    }
)["diff_of_diffs"].mean()

dff_point_diff = dff.groupby(["start_winning_team", "game_id"]).agg(
    {
        "diff_of_diffs": "min",
        "start_winning_team_won": "min"
    }
).reset_index()

sort_order = dff.groupby(["start_winning_team", "game_id"]).agg({
    "diff_of_diffs": "min"
}).reset_index().groupby("start_winning_team").agg({
    "diff_of_diffs": "mean"
}).sort_values("diff_of_diffs", ascending=False).index

fig = go.Figure()
# fig.add_hline(
#     y=0,
#     line=dict(
#         dash="dot",
#         color="black"
#     ),
#     layer="between"
# )
for t in dff["start_winning_team"].unique():
    dff_team = dff_point_diff.loc[dff_point_diff["start_winning_team"]==t]
    lower_quintile = get_percentile(dff_team.loc[dff_team["start_winning_team"]==t, "diff_of_diffs"].values, 0.25)
    upper_quintile = get_percentile(dff_team.loc[dff_team["start_winning_team"]==t, "diff_of_diffs"].values, 0.75)
    y_vals = dff_team.loc[
        (dff_team["diff_of_diffs"] < lower_quintile) | (
            dff_team["diff_of_diffs"] > upper_quintile),
        ["diff_of_diffs", "start_winning_team_won"]
    ]
    y = y_vals["diff_of_diffs"].values
    x = [t]*len(y)
    color = np.where(pd.isnull(y_vals["start_winning_team_won"]), "red", "green")
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode="markers",
            marker=dict(color=color, opacity=0.5)
        )
    )

fig.add_hline(
    y=average,
    line=dict(dash="dot", color="gray"),
    opacity=0.7
)
fig.add_annotation(
    xref="paper",
    x=1.05,
    y=average+4,
#     text=average,
    text=f"<b>Average:<br>{round(average,2)}</b>",
    font=dict(size=10),
    showarrow=False
)

fig.add_trace(
    go.Box(
        x=dff_point_diff["start_winning_team"],
        y=dff_point_diff["diff_of_diffs"],
        boxpoints=False,
        boxmean=True,
        line_color='black',
        fillcolor='lightseagreen',
        opacity=0.3,
        showwhiskers=False,
    )
)
fig.add_shape(
    type="rect",
    xref="paper",
    x0=.135,x1=.159,
    y0=-35, y1=35,
    line=dict(color="purple"), 
    opacity=0.6
)

fig.update_layout(
    xaxis={'categoryarray':sort_order},
    plot_bgcolor="white",
    showlegend=False,
    title="Point Differential Distribution After 20-point Lead - '23-24 & '24-25"
)

In [114]:
dff.loc[
    (dff["start_winning_team"] == "CHI")
    & (dff["end_winning_team"] != "CHI")
    & (dff["diff_of_diffs"] == -43),
]

,game_id,scoreHome,scoreAway,point_diff,seconds_elapsed,home_team,away_team,actionNumber,lead_20_actionNumber,lead_20_seconds_elapsed,tied_lost_lead_game_id,within_2_possessions_game_id,clutch_definition_game_id,tied_lost_lead_actionNumber,tied_lost_lead_seconds_elapsed,seconds_to_tied_lost_lead,within_2_possessions_actionNumber,within_2_possessions_seconds_elapsed,seconds_to_within_2_possessions,clutch_definition_actionNumber,clutch_definition_seconds_elapsed,seconds_to_clutch_definition,end_actionNumber,start_point_diff,end_point_diff,start_winning_team,end_winning_team,diff_of_diffs,start_winning_team_won
77968,0022400745,0.0,0.0,0.0,0,CHI,GSW,2,363,1586,NaN,0022400745,0022400745,NaN,NaN,NaN,462.0,1967.0,381.0,466.0,1994.0,408.0,726,22.0,-21.0,CHI,GSW,-43.0,None
77969,0022400745,0.0,3.0,-3.0,15,CHI,GSW,7,363,1586,NaN,0022400745,0022400745,NaN,NaN,NaN,462.0,1967.0,381.0,466.0,1994.0,408.0,726,22.0,-21.0,CHI,GSW,-43.0,None
77970,0022400745,2.0,3.0,-1.0,30,CHI,GSW,9,363,1586,NaN,0022400745,0022400745,NaN,NaN,NaN,462.0,1967.0,381.0,466.0,1994.0,408.0,726,22.0,-21.0,CHI,GSW,-43.0,None
77971,0022400745,2.0,5.0,-3.0,42,CHI,GSW,11,363,1586,NaN,0022400745,0022400745,NaN,NaN,NaN,462.0,1967.0,381.0,466.0,1994.0,408.0,726,22.0,-21.0,CHI,GSW,-43.0,None
77972,0022400745,2.0,6.0,-4.0,65,CHI,GSW,17,363,1586,NaN,0022400745,0022400745,NaN,NaN,NaN,462.0,1967.0,381.0,466.0,1994.0,408.0,726,22.0,-21.0,CHI,GSW,-43.0,None
77973,0022400745,2.0,7.0,-5.0,65,CHI,GSW,18,363,1586,NaN,0022400745,0022400745,NaN,NaN,NaN,462.0,1967.0,381.0,466.0,1994.0,408.0,726,22.0,-21.0,CHI,GSW,-43.0,None
77974,0022400745,2.0,9.0,-7.0,102,CHI,GSW,23,363,1586,NaN,0022400745,0022400745,NaN,NaN,NaN,462.0,1967.0,381.0,466.0,1994.0,408.0,726,22.0,-21.0,CHI,GSW,-43.0,None
77975,0022400745,5.0,9.0,-4.0,111,CHI,GSW,25,363,1586,NaN,0022400745,0022400745,NaN,NaN,NaN,462.0,1967.0,381.0,466.0,1994.0,408.0,726,22.0,-21.0,CHI,GSW,-43.0,None
77976,0022400745,7.0,9.0,-2.0,139,CHI,GSW,29,363,1586,NaN,0022400745,0022400745,NaN,NaN,NaN,462.0,1967.0,381.0,466.0,1994.0,408.0,726,22.0,-21.0,CHI,GSW,-43.0,None
77977,0022400745,10.0,9.0,1.0,174,CHI,GSW,33,363,1586,NaN,0022400745,0022400745,NaN,NaN,NaN,462.0,1967.0,381.0,466.0,1994.0,408.0,726,22.0,-21.0,CHI,GSW,-43.0,None


In [115]:
dff.groupby(["start_winning_team", "game_id"]).agg({
    "diff_of_diffs": "min"
}).reset_index().groupby("start_winning_team").agg({
    "diff_of_diffs": "mean"
}).sort_values("diff_of_diffs", ascending=False).reset_index().reset_index().rename(columns={
    "start_winning_team": "Team",
    "diff_of_diffs": "Average",
    "index": "Rank"
}).assign(
    Rank=lambda row: row["Rank"]+1
)

,Rank,Team,Average
0,1,OKC,1.075949
1,2,MEM,0.515152
2,3,NYK,-0.150943
3,4,BOS,-0.275000
4,5,IND,-0.487179
5,6,CLE,-0.592593
6,7,GSW,-1.128205
7,8,PHI,-1.370370
8,9,HOU,-1.568182
9,10,MIN,-1.641509


In [116]:
dff_point_diff.loc[dff_point_diff["start_winning_team"]==el, ["diff_of_diffs"]]

NameError: name 'el' is not defined

## Time-to-20

In [117]:
sort_order = dff.groupby("start_winning_team").agg({
    "lead_20_seconds_elapsed": "mean"
}).sort_values("lead_20_seconds_elapsed", ascending=True).index

In [129]:
bos=dff.loc[dff["start_winning_team"]=="BOS"]
fig = go.Figure()
fig.add_trace(
    go.Violin(
        x=bos["start_winning_team"],
        y=bos["lead_20_seconds_elapsed"]/60,
#         boxpoints='suspectedoutliers',
        marker=dict(
            color='rgb(8,81,156)',
            outliercolor='rgba(219, 64, 82, 0.6)',
            line=dict(
                outliercolor='rgba(219, 64, 82, 0.6)',
                outlierwidth=2
                )
        ),
#         boxmean=True,
        meanline=dict(visible=True),
        points="all",
        jitter=0,
        line_color='black',
        fillcolor='lightseagreen', 
        opacity=0.6,
    )
)
fig.update_layout(
    barmode='stack', 
    xaxis={'categoryarray':sort_order},
    plot_bgcolor="white",
    yaxis=dict(range=[0,50], dtick=6)
)

## Blown Leads

### Tied or Lost Lead

In [426]:
average = viz_df["tied_lost_lead_game_id"].sum()/viz_df["game_id"].sum()

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=viz_df["start_winning_team"],
        y=viz_df["tied_lost_lead_pct"],
        orientation="v",
        marker=dict(color=np.where(viz_df["start_winning_team"]=='BOS', 'green', 'lightblue')),
        text=viz_df["tied_lost_lead_pct"],
        texttemplate="%{text: .0%}",
        textposition="outside",
        textfont=dict(size=12)
    )
)

fig.add_hline(
    y=average,
    line=dict(dash="dot"),
    opacity=0.3
)
fig.add_annotation(
    xref="paper",
    x=0.1,
    y=average+.01,
    text=f"<b>Average: {round(average*100,2)}%</b>",
    font=dict(size=12),
    showarrow=False
)

fig.update_layout(
    xaxis={'categoryorder':'total ascending'},
    yaxis=dict(visible=False),
    plot_bgcolor="white",
    title="% of 20-point leads that are blown - '23-24 & '24-25"
)

### Within 2 Possessions

In [492]:
average = viz_df["within_2_possessions_game_id"].sum()/viz_df["game_id"].sum()

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=viz_df["start_winning_team"],
        y=viz_df["within_2_possessions_pct"],
        orientation="v",
        marker=dict(color=np.where(viz_df["start_winning_team"]=='BOS', 'green', 'lightblue')),
        text=viz_df["within_2_possessions_pct"],
        texttemplate="%{text: .0%}",
        textposition="outside",
        textfont=dict(size=12)
    )
)

fig.add_hline(
    y=average,
    line=dict(dash="dot"),
    opacity=0.3
)
fig.add_annotation(
    xref="paper",
    x=0.1,
    y=average+.015,
    text=f"<b>Average: {round(average*100,2)}%</b>",
    font=dict(size=12),
    showarrow=False
)

fig.update_layout(
    xaxis={'categoryorder':'total ascending'},
    yaxis=dict(visible=False),
    plot_bgcolor="white",
    title="% of 20-point leads that become 2 possession games - '23-24 & '24-25"
)

### Clutch Definition

In [445]:
average = viz_df["clutch_definition_game_id"].sum()/viz_df["game_id"].sum()

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=viz_df["start_winning_team"],
        y=viz_df["clutch_definition_pct"],
        orientation="v",
        marker=dict(color=np.where(viz_df["start_winning_team"]=='BOS', 'green', 'lightblue')),
        text=viz_df["clutch_definition_pct"],
        texttemplate="%{text: .0%}",
        textposition="outside",
        textfont=dict(size=12),
        hovertemplate="%{x}: %{y: .2%}"
                      "<extra></extra>"
    )
)

fig.add_hline(
    y=average,
    line=dict(dash="dot"),
    opacity=0.3
)
fig.add_annotation(
    xref="paper",
    x=0.1,
    y=average+.01,
    text=f"<b>Average: {round(average*100,2)}%</b>",
    font=dict(size=12),
    showarrow=False
)

fig.update_layout(
    xaxis={'categoryorder':'total ascending'},
    yaxis=dict(visible=False),
    plot_bgcolor="white",
    title="% of 20-point leads that become clutch games - '23-24 & '24-25"
)